# Regression Modelling of Ames Housing Prices 
### A Workbook Supporting the Exploratory Analysis of the Ames Housing Dataset

**Prepared by:** Rebecca Stalley-Moores  
**Date:** 20/08/2025

---

This workbook contains the full exploratory data analysis (EDA), data cleaning, feature engineering, and hypothesis testing conducted on the Ames Housing Dataset. This analysis proceeds to develop and evaluate linear regression models to accurately predict residential property sale prices.It serves as a companion to the formal report, providing detailed code, visualizations, and intermediate steps used to derive the insights summarized in the report.

In [ ]:
# install and Import libraries
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install statsmodels
!pip install pandas
!pip install numpy
!pip install scipy

# Core data manipulation and numerical computing
import pandas as pd
import numpy as np
import os
from itertools import combinations

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn imports
from sklearn.model_selection import train_test_split, cross_validate, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, PowerTransformer, PolynomialFeatures, RobustScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score, 
                           make_scorer, cross_val_score)

# Statsmodels imports
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor, OLSInfluence

# Statistical testing
from scipy import stats
from scipy.stats import f_oneway, kruskal, shapiro

# Data type checking
from pandas.api.types import is_numeric_dtype

# IPython display
from IPython.display import display

# For setting up visuals
%matplotlib inline
sns.set(style="whitegrid")

In [ ]:
# Load the dataset
df = pd.read_csv('AmesHousing.csv')


### 1. Data Summary

#### 1.1 Size of Dataset

In [ ]:
df.shape

The dataset has 82 columns and 2930 rows

#### 1.2 Description of variables

In [ ]:
df.info()

In [ ]:
df.head()

#### 1.3 identify Possible Target Variables

In [ ]:
df[['SalePrice']].describe()

## 2. Data Cleaning and Feature Engineering

#### 2.1 Find and Replace Missing Values

In [ ]:
# Find out which columns contain missing valuespor
nulls = df.isnull().sum()[df.isnull().sum() > 0].sort_values(ascending = False)
print(nulls)
print(nulls.shape)


In [ ]:
# Remove columns with > 1465 (50%) missing values
df.drop(["Pool QC", "Misc Feature", "Alley", "Fence", "Mas Vnr Type" ], axis=1, inplace=True)

# Check number of columns is now reduced to 77
df.shape

In [ ]:
#Examine remaining null value columns and their dtypes
nulls = df.isnull().sum()[df.isnull().sum() > 0]
dtypes = df.dtypes

missing_info = pd.DataFrame({
    'Missing Values': nulls,
    'Data Type': dtypes[nulls.index]
}).sort_values(by='Missing Values', ascending=False)

missing_info


In [ ]:
# Lets look at the floats first
nulls = df.isnull().sum()[df.isnull().sum() > 0].sort_values(ascending=False)

# Filter for columns that are float dtype
float_nulls = [col for col in nulls.index if df[col].dtype == 'float64']

print("Float columns with missing values:", float_nulls)

In [ ]:
# For each float column, check unique values and plot distribution
for col in float_nulls:
    print(f"--- {col} ---")
    print("Unique values:", df[col].dropna().unique())  # show first 10 unique values
    print("Number of unique values:", df[col].nunique())
    
    # Plot distribution
    plt.figure(figsize=(6,3))
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.show()


In [ ]:
# Lets look at discrete numeric counts first
df[['Bsmt Half Bath', 'Bsmt Full Bath', 'Garage Cars']].isnull().sum()

In [ ]:
# These could be null values simply because there is no basement or garage
# Impute Bsmt Full Bath = 0 if no basement
df.loc[df['Total Bsmt SF'] == 0.0, 'Bsmt Full Bath'] = 0.0

# Then fill the rest with median
df['Bsmt Full Bath'].fillna(df['Bsmt Full Bath'].median(), inplace=True)

df.loc[df['Total Bsmt SF'] == 0.0, 'Bsmt Half Bath'] = 0.0
df['Bsmt Half Bath'].fillna(df['Bsmt Half Bath'].median(), inplace=True)

df.loc[df['Garage Area'] == 0.0, 'Garage Cars'] = 0.0
df['Garage Cars'].fillna(df['Garage Cars'].median(), inplace=True)

In [ ]:
#check discrete numeric count null values have been resolved
df[['Bsmt Half Bath', 'Bsmt Full Bath', 'Garage Cars']].isnull().sum()

In [ ]:
# For the remaining features the null value likely means that there is no garage, hence the null value for 'Garage Area'
# We can inspect against related columns to try to confirm this
# Check Garage Yr Blt null rows: are Garage Area and Garage Cars zero or null?
garage_null = df[df['Garage Yr Blt'].isnull()]
print("Garage Yr Blt NULL rows info:")
print(garage_null[['Garage Area', 'Garage Cars']].describe())
print("\nSample rows:")
print(garage_null[['Garage Area', 'Garage Cars']].head(), "\n")

# Check Mas Vnr Area null rows: is it zero or something else?
mas_vnr_null = df[df['Mas Vnr Area'].isnull()]
print("Mas Vnr Area NULL rows info:")
print(mas_vnr_null[['Mas Vnr Area']].describe())
print("\nSample rows:")
print(mas_vnr_null[['Mas Vnr Area']].head(), "\n")

# Check Total Bsmt SF null rows: what are the basement bath counts and basement areas?
bsmt_null = df[df['Total Bsmt SF'].isnull()]
print("Total Bsmt SF NULL rows info:")
print(bsmt_null[['Bsmt Full Bath', 'Bsmt Half Bath', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF']].describe())
print("\nSample rows:")
print(bsmt_null[['Bsmt Full Bath', 'Bsmt Half Bath', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF']].head(), "\n")

# Check BsmtFin SF 2 null rows: related basement features
bsmtfin2_null = df[df['BsmtFin SF 2'].isnull()]
print("BsmtFin SF 2 NULL rows info:")
print(bsmtfin2_null[['Total Bsmt SF', 'BsmtFin SF 1', 'Bsmt Unf SF']].describe())
print("\nSample rows:")
print(bsmtfin2_null[['Total Bsmt SF', 'BsmtFin SF 1', 'Bsmt Unf SF']].head(), "\n")

# Check Bsmt Unf SF null rows:
bsmtunf_null = df[df['Bsmt Unf SF'].isnull()]
print("Bsmt Unf SF NULL rows info:")
print(bsmtunf_null[['Total Bsmt SF', 'BsmtFin SF 1', 'BsmtFin SF 2']].describe())
print("\nSample rows:")
print(bsmtunf_null[['Total Bsmt SF', 'BsmtFin SF 1', 'BsmtFin SF 2']].head(), "\n")

# Check Garage Area null rows:
garage_area_null = df[df['Garage Area'].isnull()]
print("Garage Area NULL rows info:")
print(garage_area_null[['Garage Cars', 'Garage Yr Blt']].describe())
print("\nSample rows:")
print(garage_area_null[['Garage Cars', 'Garage Yr Blt']].head(), "\n")

# Check Lot Frontage null rows: see if there is a pattern or related features to check
lot_frontage_null = df[df['Lot Frontage'].isnull()]
print("Lot Frontage NULL rows info:")
print(lot_frontage_null.describe(include='all'))
print("\nSample rows:")
print(lot_frontage_null.head(), "\n")


In [ ]:
# Lot Frontage appears to have genuine missing data whereas the remainder appear to be null because the feature doesn't exist
# 1. Columns where NULL means "feature doesn't exist" — fill with 0 or suitable default:

# Basement-related features:
bsmt_cols = [
    'Total Bsmt SF', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF'
]

for col in bsmt_cols:
    df[col].fillna(0, inplace=True)

# Garage-related features:
garage_cols = ['Garage Yr Blt', 'Garage Area']
# Garage Year Built — replace null with 0 or a special indicator like 0 to mean 'No Garage'
df['Garage Yr Blt'].fillna(0, inplace=True)
df['Garage Area'].fillna(0, inplace=True)

# Masonry veneer area — fill NaN with 0 (no veneer)
df['Mas Vnr Area'].fillna(0, inplace=True)

# 2. Lot Frontage — actual missing data, so use median overall here.
median_lot_frontage = df['Lot Frontage'].median()
df['Lot Frontage'].fillna(median_lot_frontage, inplace=True)


In [ ]:
# Check float nulls have been removed
nulls = df.isnull().sum()[df.isnull().sum() > 0].sort_values(ascending=False)

# Filter for columns that are float dtype
float_nulls = [col for col in nulls.index if df[col].dtype == 'float64']
float_nulls

In [ ]:
# Now lets look at categorical nulls
cat_nulls = [col for col in nulls.index if df[col].dtype == 'object']

print("Categorical columns with missing values:", cat_nulls)

In [ ]:
# For each categorical column, check unique values and plot distribution
for col in cat_nulls:
    print(f"--- {col} ---")
    print("Unique values:", df[col].dropna().unique())  # show first 10 unique values
    print("Number of unique values:", df[col].nunique())
    
    # Plot distribution
    plt.figure(figsize=(6,3))
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.show()

In [ ]:
# The above suggest that for all features apart from Electrical, missing data implies the feature doesn't exist
# lets check this before imputing
none_fill_map = {
    'Fireplace Qu': df['Fireplaces'] == 0,
    'Garage Type': (df['Garage Area'] == 0) & (df['Garage Cars'] == 0),
    'Garage Finish': (df['Garage Area'] == 0) & (df['Garage Cars'] == 0),
    'Garage Qual': (df['Garage Area'] == 0) & (df['Garage Cars'] == 0),
    'Garage Cond': (df['Garage Area'] == 0) & (df['Garage Cars'] == 0),
    'Bsmt Exposure': df['Total Bsmt SF'] == 0,
    'BsmtFin Type 1': df['Total Bsmt SF'] == 0,
    'BsmtFin Type 2': df['Total Bsmt SF'] == 0,
    'Bsmt Qual': df['Total Bsmt SF'] == 0,
    'Bsmt Cond': df['Total Bsmt SF'] == 0,
}

# Fill 'None' where the missing is due to absence of the feature
for col, condition in none_fill_map.items():
    df.loc[condition & df[col].isnull(), col] = 'None'

# Now double-check and fill any remaining NaNs in these columns just in case
for col in none_fill_map.keys():
    df[col].fillna('None', inplace=True)

# Fill Electrical with mode (likely just missing info)
df['Electrical'].fillna(df['Electrical'].mode()[0], inplace=True)


In [ ]:
# Check categorical nulls have been removed
nulls = df.isnull().sum()[df.isnull().sum() > 0].sort_values(ascending=False)
nulls


#### 2.2 Check Data Types

In [ ]:
# Convert Discrete Numeric Counts to Integers
df["Bsmt Half Bath"] = df["Bsmt Half Bath"].astype(int)
df["Bsmt Full Bath"] = df["Bsmt Full Bath"].astype(int)
df["Garage Cars"] = df["Garage Cars"].astype(int)

In [ ]:
df[["Bsmt Half Bath", "Bsmt Full Bath","Garage Cars"]].head()

#### 2.3 Check for duplicate rows

In [ ]:
df['PID'].nunique()

In [ ]:
# Therefore no duplicated rows

#### 2.4 Check for Inconsistent Categorical Data

In [ ]:
# Already checked above but to make sure:
cat_feats = df.select_dtypes(include='object')

#### 2.5 Check for Outliers in Numerical Data


In [ ]:
print(df.describe()) 
num_feats = df.select_dtypes(include='number')

batch_size = 10
n_cols = 3

num_feats = df.select_dtypes(include='number')
n_features = len(num_feats.columns)

for batch_start in range(0, n_features, batch_size):
    batch = num_feats.columns[batch_start:batch_start + batch_size]
    n = len(batch)
    n_rows = (n + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
    axes = axes.flatten()

    for i, feature in enumerate(batch):
        sns.boxplot(data=df, x=feature, ax=axes[i])
        axes[i].set_title(f'Boxplot of {feature}')
        axes[i].set_xlabel(feature)

    # Remove unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()

    # Save the figure with batch index (starting from 1)
    batch_num = (batch_start // batch_size) + 1
    plt.savefig(f'boxplots_batch_{batch_num}.png')
    plt.show()



In [ ]:
num_feats = df.select_dtypes(include='number').columns
skew_threshold = 1.0

log_transform_cols = []

for col in num_feats:
    skewness = df[col].skew()
    if skewness > skew_threshold:
        log_transform_cols.append(col)
        # Create new column with suffix '_log'
        df[col + '_log'] = np.log1p(df[col])

print("Log transformed columns saved as new features:")
print(log_transform_cols)



In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
print(df.describe())
log_transformed = [
    "MS SubClass_log",
    "Lot Frontage_log",
    "Lot Area_log",
    "Mas Vnr Area_log",
    "BsmtFin SF 1_log",
    "BsmtFin SF 2_log",
    "Total Bsmt SF_log",
    "1st Flr SF_log",
    "Low Qual Fin SF_log",
    "Gr Liv Area_log",
    "Bsmt Half Bath_log",
    "Kitchen AbvGr_log",
    "Wood Deck SF_log",
    "Open Porch SF_log",
    "Enclosed Porch_log",
    "3Ssn Porch_log",
    "Screen Porch_log",
    "Pool Area_log",
    "Misc Val_log",
    "SalePrice_log"
]

transformed_feats = df[log_transformed]

batch_size = 10
n_cols = 3

n_features = len(transformed_feats.columns)

for batch_start in range(0, n_features, batch_size):
    batch = transformed_feats.columns[batch_start:batch_start + batch_size]
    n = len(batch)
    n_rows = (n + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
    axes = axes.flatten()

    for i, feature in enumerate(batch):
        sns.boxplot(data=df, x=feature, ax=axes[i])
        axes[i].set_title(f'Boxplot of {feature}')
        axes[i].set_xlabel(feature)

    # Remove unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()

    # Save the figure with batch index (starting from 1)
    batch_num = (batch_start // batch_size) + 1
    plt.savefig(f'transformed boxplots_batch_{batch_num}.png')
    plt.show()



In [ ]:
# The log transformation has not really improved the outliers so trying Yeo-Johnson transform next as often performs better when 
# there are a lot of small values, including zeros and negatives

# Set skewness threshold
skew_threshold = 0.75

# Select numerical features
num_feats = df.select_dtypes(include='number')

# Calculate skewness
skew_vals = num_feats.skew().sort_values(ascending=False)

# Identify features with high skew
high_skew_feats = skew_vals[abs(skew_vals) > skew_threshold].index.tolist()

# Apply Yeo-Johnson transform
pt = PowerTransformer(method='yeo-johnson')

for feature in high_skew_feats:
    reshaped_data = df[[feature]].values  # 2D array for sklearn
    try:
        transformed = pt.fit_transform(reshaped_data)
        new_col_name = f"{feature}_yj"
        df[new_col_name] = transformed
    except Exception as e:
        print(f"Could not transform {feature}: {e}")

# List of new columns created
transformed_yj = [f"{feature}_yj" for feature in high_skew_feats]
print("Yeo-Johnson transformed features:", transformed_yj)



In [ ]:
df.shape

In [ ]:
yj_feats = df[transformed_yj]
batch_size = 10
n_cols = 3
n_features = len(yj_feats.columns)

for batch_start in range(0, n_features, batch_size):
    batch = yj_feats.columns[batch_start:batch_start + batch_size]
    n = len(batch)
    n_rows = (n + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
    axes = axes.flatten()

    for i, feature in enumerate(batch):
        sns.boxplot(data=df, x=feature, ax=axes[i])
        axes[i].set_title(f'Boxplot of {feature}')
        axes[i].set_xlabel(feature)

    # Remove unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()

    # Save the figure with batch index (starting from 1)
    batch_num = (batch_start // batch_size) + 1
    plt.savefig(f'transformed boxplots_batch_{batch_num}.png')
    plt.show()

In [ ]:
df[df.columns[-35:]].describe()

In [ ]:
# This hasn't helped and it has made the data less interpretable as a whole.  
# Capping will be used to reduce skew and influence of rare cases.

# Define features to cap with their capping percentiles
capping_config = {
    'Lot Area': 0.99,
    'Mas Vnr Area': 0.99,
    'Wood Deck SF': 0.99,
    'Open Porch SF': 0.99,
    'Enclosed Porch': 0.99,
    '3Ssn Porch': 0.95,
    'Screen Porch': 0.95,
    'Pool Area': 0.95,
    'Misc Val': 0.99,
    'Gr Liv Area': 0.99,
    'SalePrice': 0.99  
}

# Create capped versions of features
for feature, quantile in capping_config.items():
    cap_value = df[feature].quantile(quantile)
    new_col = f"{feature}_capped"
    df[new_col] = np.where(df[feature] > cap_value, cap_value, df[feature])
    print(f"Capped '{feature}' at {quantile*100:.0f}th percentile: {cap_value:.2f} → New column: '{new_col}'")


In [ ]:
capped_columns = [
    "Lot Area_capped",
    "Mas Vnr Area_capped",
    "Wood Deck SF_capped",
    "Open Porch SF_capped",
    "Enclosed Porch_capped",
    "3Ssn Porch_capped",
    "Screen Porch_capped",
    "Pool Area_capped",
    "Misc Val_capped",
    "Gr Liv Area_capped",
    "SalePrice_capped"
]
capped_feats = df[capped_columns]
batch_size = 10
n_cols = 3
n_features = len(capped_feats.columns)

for batch_start in range(0, n_features, batch_size):
    batch = capped_feats.columns[batch_start:batch_start + batch_size]
    n = len(batch)
    n_rows = (n + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
    axes = axes.flatten()

    for i, feature in enumerate(batch):
        sns.boxplot(data=df, x=feature, ax=axes[i],color="green")
        axes[i].set_title(f'Boxplot of {feature}')
        axes[i].set_xlabel(feature)

    # Remove unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()

    # Save the figure with batch index (starting from 1)
    batch_num = (batch_start // batch_size) + 1
    plt.savefig(f'transformed boxplots_batch_{batch_num}.png')
    plt.show()

#### 2.6 Logical Inconsistencies


In [ ]:
# Logical Inconsistency Checks

# 1. Basement area is 0, but other basement features exist
basement_inconsistencies = df[
    (df["Total Bsmt SF"] == 0) & (
        (df["BsmtFin SF 1"] > 0) | 
        (df["BsmtFin SF 2"] > 0) | 
        (df["Bsmt Full Bath"] > 0) | 
        (df["Bsmt Half Bath"] > 0)
    )
]

# 2. Garage area is 0, but garage features are filled
garage_inconsistencies = df[
    (df["Garage Area"] == 0) & (
        (df["Garage Cars"] > 0) |
        (df["Garage Yr Blt"] > 0)
    )
]

# 3. Remodel year before the build year
remodel_inconsistencies = df[
    df["Year Remod/Add"] < df["Year Built"]
]

# 4. 2nd floor area without 1st floor area
second_floor_inconsistencies = df[
    (df["2nd Flr SF"] > 0) & (df["1st Flr SF"] == 0)
]

# 5. Gross living area smaller than sum of 1st + 2nd + low quality finished SF
living_area_inconsistencies = df[
    df["Gr Liv Area"] < (df["1st Flr SF"] + df["2nd Flr SF"] + df["Low Qual Fin SF"])
]

# Display summary of findings
print("Basement inconsistencies:", len(basement_inconsistencies))
print("Garage inconsistencies:", len(garage_inconsistencies))
print("Remodel year inconsistencies:", len(remodel_inconsistencies))
print("2nd floor inconsistencies:", len(second_floor_inconsistencies))
print("Living area inconsistencies:", len(living_area_inconsistencies))


In [ ]:
print(df.loc[df['Order'] == 2237, ['Garage Cars', 'Garage Yr Blt']])
garage_inconsistencies

In [ ]:
# The garage features are contradictory here so will delete this row - it is the only garage inconsisteny and this is not a very small dataset
df = df[df['Order'] != 2237]

In [ ]:
# There is 1 house where Year Remod/Add is before Year Built, which is illogical
remodel_inconsistencies

In [ ]:
# Impossible to know if original build year or remodel year is correct so this one row will be dropped
df = df[df['Order'] != 851]

In [ ]:
# For Living Area inconistency there are too many rows to remove so we will flag these rows for later consideration wen modelling
df['living_area_inconsistency'] = (
    df['Gr Liv Area'] < (df['1st Flr SF'] + df['2nd Flr SF'] + df['Low Qual Fin SF'])
).astype(int)

#### 2.7 Unusual String Lengths

In [ ]:
string_cols = df.select_dtypes(include='object').columns

for col in string_cols:
    lengths = df[col].apply(len)
    
    print(f"Column: {col}")
    print(f"  Min length: {lengths.min()}")
    print(f"  Max length: {lengths.max()}")
    print(f"  Mean length: {lengths.mean():.2f}")
    print(f"  95th percentile length: {lengths.quantile(0.95)}")
    
    threshold = lengths.quantile(0.95)
    unusual = df[lengths > threshold][col]
    
    print(f"  Examples of long strings (> 95th percentile): {unusual.head(5).to_list()}\n")


In [ ]:
# No unusual string lengths found

#### 2.8 Blank strings

In [ ]:
string_cols = df.select_dtypes(include='object').columns
found_blanks = False

for col in string_cols:
    blank_count = df[col].apply(lambda x: isinstance(x, str) and x.strip() == '').sum()
    if blank_count > 0:
        print(f"Column '{col}' has {blank_count} blank/whitespace-only strings.")
        found_blanks = True

if not found_blanks:
    print("No blank or whitespace-only strings found in any string column.")


#### 2.9 Create new features

In [ ]:
# Age - related features
df['house_age'] = df['Yr Sold'] - df['Year Built']
df['remodel_age'] = df['Yr Sold'] - df['Year Remod/Add']
df['years_to_remodel'] = df['Year Remod/Add'] - df['Year Built']

# Combine all full and half bathrooms (both basement and above grade) into one feature
df['total_bathrooms'] = (df['Bsmt Full Bath'] + 0.5 * df['Bsmt Half Bath'] + 
                         df['Full Bath'] + 0.5 * df['Half Bath'])

# Sum all porch areas (Wood Deck, Open Porch, Enclosed Porch, 3Ssn Porch, Screen Porch) to get total outdoor living space
df['total_porch_area'] = (df['Wood Deck SF'] + df['Open Porch SF'] + df['Enclosed Porch'] + 
                          df['3Ssn Porch'] + df['Screen Porch'])

# Sum of finished basement areas (type 1 + type 2)
df['total_bsmt_finished'] = df['BsmtFin SF 1'] + df['BsmtFin SF 2']

# Ratio of above ground living area to total lot area — gives sense of density
df['living_area_ratio'] = df['Gr Liv Area'] / df['Lot Area']

# Combine Overall Qual and Overall Cond into a single "quality score": average or weighted sum
df['avg_quality'] = df[['Overall Qual', 'Overall Cond']].mean(axis=1)

# From Mo Sold, create seasons or quarters (Winter, Spring, Summer, Fall)
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3,4,5]:
        return 'Spring'
    elif month in [6,7,8]:
        return 'Summer'
    else:
        return 'Fall'

df['season_sold'] = df['Mo Sold'].apply(get_season)

# Interaction of Overall Qual and Gr Liv Area
df['qual_living_area_interaction'] = df['Overall Qual'] * df['Gr Liv Area']

In [ ]:
df.columns.to_list()

In [ ]:
# We need to create a fresh copy of the df without the log transformed and yeo-johnson transformed columns
df_no_log_yj = df[[col for col in df.columns if '_log' not in col and '_yj' not in col]].copy()


In [ ]:
df_no_log_yj.columns.to_list()

In [ ]:
# Three of our created features were based on original features rather than capped features in error.  Can fix this now:
# ✅ Recreate using _capped columns

# Recalculate total_porch_area using capped values
df['total_porch_area'] = (
    df['Wood Deck SF_capped'] + df['Open Porch SF_capped'] + 
    df['Enclosed Porch_capped'] + df['3Ssn Porch_capped'] + 
    df['Screen Porch_capped']
)

# Recalculate living_area_ratio using capped columns
df['living_area_ratio'] = df['Gr Liv Area_capped'] / df['Lot Area_capped']

# Recalculate qual_living_area_interaction using capped Gr Liv Area
df['qual_living_area_interaction'] = df['Overall Qual'] * df['Gr Liv Area_capped']

In [ ]:
# Also to be fixed in df_no_log_yj

# Recalculate total_porch_area using capped values
df_no_log_yj['total_porch_area'] = (
    df_no_log_yj['Wood Deck SF_capped'] + df_no_log_yj['Open Porch SF_capped'] + 
    df_no_log_yj['Enclosed Porch_capped'] + df_no_log_yj['3Ssn Porch_capped'] + 
    df_no_log_yj['Screen Porch_capped']
)

# Recalculate living_area_ratio using capped columns
df_no_log_yj['living_area_ratio'] = df_no_log_yj['Gr Liv Area_capped'] / df_no_log_yj['Lot Area_capped']

# Recalculate qual_living_area_interaction using capped Gr Liv Area
df_no_log_yj['qual_living_area_interaction'] = df_no_log_yj['Overall Qual'] * df_no_log_yj['Gr Liv Area_capped']


In [ ]:
# Remove original columns were capped versions exist and rename df
# List of capped columns suffixes (without _capped)
capped_cols_togo = [
    'Lot Area',
    'Mas Vnr Area',
    'Wood Deck SF',
    'Open Porch SF',
    'Enclosed Porch',
    '3Ssn Porch',
    'Screen Porch',
    'Pool Area',
    'Misc Val',
    'Gr Liv Area',
    'SalePrice'
]

# Create list of original columns to drop (those that have capped versions)
cols_to_drop = capped_cols_togo

# Drop these original columns from df_no_log_yj
df_no_log_yj = df_no_log_yj.drop(columns=cols_to_drop)

# Rename df_no_log_yj to df_new
df_new = df_no_log_yj.copy()


## 3. EDA

#### 3.1 Univariate Analysis

#### 3.1.1 Numerical Features

In [ ]:
df_new.select_dtypes(include='number').shape

In [ ]:
# As there are 49 numerical columns lets remove those with little variance as these are not very informative
# Select numerical columns only from df_final
num_cols = df_new.select_dtypes(include=[np.number]).columns

# Calculate variance for each numerical feature
variances = df_new[num_cols].var()

# Set a threshold for "low variance" (adjust as needed)
low_variance_threshold = 0.01

# Identify features with variance below the threshold
low_variance_features = variances[variances < low_variance_threshold].index.tolist()

print("Features with low variance:")
print(low_variance_features)

In [ ]:
num_cols = [col for col in num_cols if col not in ["3Ssn Porch_capped", "Pool Area_capped"]]
len(num_cols)
                         

In [ ]:
# Step 1: Get all numeric columns
num_cols_all = df_new.select_dtypes(include=[np.number]).columns.tolist()

# Step 2: Define ID-like columns
id_cols = ['Order', 'PID']

# Step 3: Define ordinal columns (manually identified from domain knowledge)
ordinal_cols = [
    'Overall Qual', 'Overall Cond', 'Exter Qual', 'Exter Cond',
    'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1',
    'BsmtFin Type 2', 'Heating QC', 'Kitchen Qual', 'Functional',
    'Fireplace Qu', 'Garage Finish', 'Garage Qual', 'Garage Cond',
    'Paved Drive'
]

# Only keep those that are present in df_new
ordinal_cols = [col for col in ordinal_cols if col in df_new.columns]

# Step 4: Combine all exclusions
cols_to_exclude = set(id_cols + ordinal_cols)

# Step 5: Filter numerical columns for EDA
num_cols_filtered = [col for col in num_cols_all if col not in cols_to_exclude]

# Show results
print("✅ Excluded columns:")
print(sorted(cols_to_exclude))
print(f"\n✅ Final numerical columns for univariate analysis ({len(num_cols_filtered)}):")
print(sorted(num_cols_filtered))


In [ ]:
output_dir = "eda_univariate_plots"
os.makedirs(output_dir, exist_ok=True)

cols = num_cols_filtered  # or whatever your final list is
n_cols = 3  # plots per row
n_rows = (len(cols) + n_cols - 1) // n_cols  # ceil division

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4))
axes = axes.flatten()

# Plot each histogram with KDE and save it
for i, col in enumerate(cols):
    ax = axes[i]
    sns.histplot(data=df_new, x=col, kde=True, ax=ax, bins=30, color='skyblue', edgecolor='black')
    ax.set_title(f'{col}', fontsize=10)

    # Save each plot as its own figure too
    single_fig = plt.figure(figsize=(6, 4))
    sns.histplot(data=df_new, x=col, kde=True, bins=30, color='skyblue', edgecolor='black')
    plt.title(f'{col} Distribution')
    plt.tight_layout()
    single_fig.savefig(f"{output_dir}/{col.replace('/', '_')}_hist_kde.png")
    plt.close(single_fig)  # Close to avoid clutter

# Remove empty subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [ ]:
# Compute correlation with the target
correlations = df_new[cols].corr()['SalePrice_capped'].drop('SalePrice_capped')

# Sort by absolute correlation value (descending)
sorted_corr = correlations.abs().sort_values(ascending=False)

# Display top correlations (you can adjust the number)
sorted_corr_top = correlations.loc[sorted_corr.index]
print("Correlations with SalePrice_capped:\n")
display(sorted_corr_top)


In [ ]:
# Create output directory if it doesn't exist
output_dir = "eda_boxplots"
os.makedirs(output_dir, exist_ok=True)

# Number of plots per row
n_cols = 3
n_rows = (len(cols) + n_cols - 1) // n_cols  # ceil division

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(cols):
    ax = axes[i]
    
    # Create horizontal boxplot in the grid
    sns.boxplot(data=df_new, x=col, ax=ax, color='lightblue')
    ax.set_title(f'{col}', fontsize=10)

    # Save individual horizontal boxplot
    single_fig = plt.figure(figsize=(6, 4))  # Wider than tall for horizontal
    sns.boxplot(data=df_new, x=col, color='lightblue')
    plt.title(f'{col} Boxplot')
    plt.tight_layout()
    single_fig.savefig(f"{output_dir}/{col.replace('/', '_')}_boxplot.png")
    plt.close(single_fig)

# Remove any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


#### 3.1.2 Categorical Analysis

In [ ]:
# Make sure we capture all categorical features
# 1. Identify all object and category dtype columns
df_new['MS SubClass'] = df_new['MS SubClass'].astype(str)
cat_cols = df_new.select_dtypes(include=['object', 'category']).columns.tolist()

# 2. Manually check for any int/float columns that are actually categorical (e.g. encoded labels, discrete small-range integers)
# For example, identify numeric columns with few unique values
potential_cats = [col for col in df_new.select_dtypes(include=['int64', 'float64']).columns
                  if df_new[col].nunique() < 10]  # You can adjust this threshold

# 3. Combine with known categorical types
cat_cols_final = cat_cols + potential_cats
cat_cols_final
# 4. Optionally remove any that are clearly numeric (e.g. total bathrooms)
# You may want to filter further based on domain knowledge


In [ ]:
# Manually remove known numerical features
to_remove = [
    'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath',
    'Bedroom AbvGr', 'Kitchen AbvGr', 'Fireplaces', 'Garage Cars',
    'Yr Sold', '3Ssn Porch_capped', 'Pool Area_capped',
    'living_area_inconsistency'
]

# Remove them from your cat_cols_final list
cat_cols_final = [col for col in cat_cols_final if col not in to_remove]
cat_cols_final

In [ ]:
# Set up output folder
output_dir = "eda_catplots"
os.makedirs(output_dir, exist_ok=True)

# Layout for subplots
n_cols = 3
n_rows = (len(cat_cols_final) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 6, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(cat_cols_final):
    ax = axes[i]
    
    # Create countplot in the subplot grid
    sns.countplot(data=df_new, x=col, ax=ax, order=df_new[col].value_counts().index, palette='pastel')
    ax.set_title(col)
    ax.tick_params(axis='x', rotation=45)

    # Save individual figure
    single_fig = plt.figure(figsize=(8, 4))
    sns.countplot(data=df_new, x=col, order=df_new[col].value_counts().index, palette='pastel')
    plt.title(f'{col} Count Plot')
    plt.xticks(rotation=45)
    plt.tight_layout()
    single_fig.savefig(f"{output_dir}/{col.replace('/', '_')}_countplot.png")
    plt.close(single_fig)

# Delete any unused axes
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [ ]:
len(cat_cols_final)

In [ ]:
# As a large number of features - need to find the most relevant features to present in written report

# Bin SalePrice for categorical association
df_new['SalePrice_bin'] = pd.qcut(df_new['SalePrice_capped'], q=4, duplicates='drop')

anova_results = {}
for col in cat_cols_final:
    groups = [df_new.loc[df_new[col] == cat, 'SalePrice_capped'] for cat in df_new[col].dropna().unique()]
    try:
        f_stat, p_val = f_oneway(*groups)
        anova_results[col] = p_val
    except:
        anova_results[col] = np.nan  # For features with too few groups

# Sort by significance (lowest p-values)
sorted_anova = dict(sorted(anova_results.items(), key=lambda item: item[1] if item[1] is not None else 1))
print(sorted_anova)


In [ ]:
# keep all features apart from utilities
cat_cols_final = [col for col in cat_cols_final if col != 'Utilities']

In [ ]:
# drop features with no variability
remove_cols = ['Street', 'Condition 2', 'Roof Matl']
cat_cols_final = [col for col in cat_cols_final if col not in remove_cols]



In [ ]:
# Check further for low variability features
threshold = 0.90

low_variance_cols = []

for col in cat_cols_final:
    # Get normalized value counts (proportion of each category)
    top_freq = df_new[col].value_counts(normalize=True).iloc[0]
    
    if top_freq > threshold:
        low_variance_cols.append(col)

print("Low variance categorical columns (remove or consider removing):")
print(low_variance_cols)

# Remove low variance columns from your categorical features list
cat_cols_reduced = [col for col in cat_cols if col not in low_variance_cols]

In [ ]:
# drop features with little variability
remove_cols = ['Land Slope','Heating','Central Air','Functional','Electrical','Garage Cond', 'Paved Drive']
cat_cols_final = [col for col in cat_cols_final if col not in remove_cols]
cat_cols_final

In [ ]:
anova_sorted = {
    'Neighborhood': 0.0,
    'Exter Qual': 0.0,
    'Bsmt Qual': 0.0,
    'Kitchen Qual': 0.0,
    'Garage Finish': 1.72e-248,
    'Fireplace Qu': 1.14e-240,
    'Foundation': 1.89e-219,
    'Garage Type': 1.25e-191,
    'MS SubClass': 7.73e-180,         # Note: MS SubClass is categorical but omitted from cat_cols_final below
    'BsmtFin Type 1': 2.60e-169,
    'Heating QC': 9.55e-162,
    'Bsmt Exposure': 1.34e-124,
    'Exterior 1st': 1.73e-116,
    'Exterior 2nd': 3.31e-113,
    'Overall Cond': 3.58e-105,
    'Sale Condition': 9.14e-97,
    'Sale Type': 1.69e-93,
    'MS Zoning': 4.68e-81,
    'Lot Shape': 8.74e-64,
    'Garage Qual': 2.51e-61,
    'Garage Cond': 8.09e-57,
    'Paved Drive': 2.01e-55,
    'House Style': 7.66e-50,
    'Roof Style': 1.91e-47,
    'Bsmt Cond': 2.90e-34,
    'Land Contour': 9.97e-29,
    'Condition 1': 2.81e-26,
    'Bldg Type': 7.05e-23,
    'BsmtFin Type 2': 1.02e-18,
    'Exter Cond': 4.71e-18,
    'Lot Config': 2.46e-12,
    'season_sold': 0.00147
}

cat_cols_final = [
 'MS Zoning',
 'Lot Shape',
 'Land Contour',
 'Lot Config',
 'Neighborhood',
 'Condition 1',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Exterior 1st',
 'Exterior 2nd',
 'Exter Qual',
 'Exter Cond',
 'Foundation',
 'Bsmt Qual',
 'Bsmt Cond',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'BsmtFin Type 2',
 'Heating QC',
 'Kitchen Qual',
 'Fireplace Qu',
 'Garage Type',
 'Garage Finish',
 'Garage Qual',
 'Sale Type',
 'Sale Condition',
 'season_sold',
 'Overall Cond',
 'MS SubClass'
]

# Filter anova_sorted to only include features in cat_cols_final
anova_filtered = {k: v for k, v in anova_sorted.items() if k in cat_cols_final}

# Sort filtered features by p-value ascending
sorted_features = sorted(anova_filtered, key=anova_filtered.get)

# Top 15 features by ANOVA p-value:
top_15_features = sorted_features[:15]

print(top_15_features)


In [ ]:
cat_cols_final = top_15_features

In [ ]:
# Domain knowledge features to add back
domain_features = [
    'MS Zoning', 'House Style', 'Condition 1', 'Roof Style', 'Paved Drive, '
]

for feature in domain_features:
    if feature not in cat_cols_final:
        cat_cols_final.append(feature)
cat_cols_final

#### 3.2 Bivariate Analysis
#### 3.2.1 Numerical Versus Numerical

In [ ]:
num_cols_filtered = [col for col in num_cols_filtered if is_numeric_dtype(df_new[col])]
# Regplots for all numerical features
output_dir = 'numeric_vs_target'
os.makedirs(output_dir, exist_ok=True)

plots_per_row = 3
total_plots = len(num_cols_filtered)
rows = (total_plots + plots_per_row - 1) // plots_per_row

fig, axes = plt.subplots(rows, plots_per_row, figsize=(plots_per_row * 5, rows * 4))
axes = axes.flatten()

for i, feature in enumerate(num_cols_filtered):
    ax = axes[i]
    sns.regplot(data=df_new, x=feature, y='SalePrice_capped', ax=ax,
            scatter_kws={'alpha':0.1,'color': 'mediumblue'}, line_kws={'color':'red'})
    ax.set_title(f'{feature} vs SalePrice_capped', fontsize=10)
    ax.set_xlabel(feature)
    ax.set_ylabel('SalePrice_capped')

    # Save individual plot
    single_fig = plt.figure(figsize=(6, 4))
    sns.regplot(data=df_new, x=feature, y='SalePrice_capped',
            scatter_kws={'alpha':0.1, 'color': 'mediumblue'}, line_kws={'color':'red'})
    plt.title(f'{feature} vs SalePrice_capped')
    plt.tight_layout()
    safe_feature = feature.replace('/', '_').replace('\\', '_').replace(':', '_')
    single_fig.savefig(f"{output_dir}/{safe_feature}_vs_SalePrice_capped.png")
    plt.close(single_fig)

# Remove empty subplots
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

fig.tight_layout()
plt.show()


In [ ]:
# Heatmap for all numerical features
# use select features being shown in written report and group thematically
groups = {
    "Size & Area Features": [
        'Gr Liv Area_capped',
        '1st Flr SF',
        'Total Bsmt SF',
        'Mas Vnr Area_capped',
        'Lot Area_capped',
        'Wood Deck SF_capped',
        'total_bsmt_finished',
        'qual_living_area_interaction'
    ],
    "Bathrooms & Plumbing": [
        'Full Bath',
        'total_bathrooms'
    ],
    "Rooms & Interior Count": [
        'TotRms AbvGrd',
        'Bedroom AbvGr',
        'Kitchen AbvGr'
    ],
    "Quality / Composite Scores": [
        'avg_quality',
        'qual_living_area_interaction'
    ],
    "Age / Time-Based Features": [
        'house_age',
        'remodel_age',
        'Yr Sold'
    ],
    "Garage, Porch and External Features": [
        'Garage Area',
        'Open Porch SF_capped',
        'total_porch_area'
    ],
   
    "Structural / Other": [
        'Fireplaces'
    ]
}

target = 'SalePrice_capped'
output_dir = 'heatmaps'
os.makedirs(output_dir, exist_ok=True)
vmin = -1
vmax = 1


def plot_and_save_group_corr_heatmap(df, features, group_name, target=target, cmap='viridis', output_dir=output_dir):
    cols = features + [target]
    corr = df[cols].corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))

    plt.figure(figsize=(8, 6))
    sns.heatmap(
        corr,
        mask=mask,
        annot=True,
        fmt='.2f',
        cmap=cmap,
        center=0,
        vmin=0,          # Fix color range: start at 0
        vmax=1,          # End at 1 for perfect correlation
        square=True,
        linewidths=0.5,
        cbar_kws={'shrink': 0.7},
        annot_kws={"size": 8}
    )
    plt.title(f'Correlation Heatmap: {group_name}', fontsize=14)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()
    safe_group_name = group_name.replace(' ', '_').replace('/', '_').replace('\\', '_')
    save_path = f"{output_dir}/{safe_group_name}_corr_heatmap.png"
    plt.savefig(save_path)
    plt.show()
    plt.close()
    print(f"Saved heatmap: {save_path}")

for group_name, features in groups.items():
    filtered_features = [f for f in features if f in df_new.columns]
    if filtered_features:
        print(f"Plotting and saving group: {group_name} ({len(filtered_features)} features)")
        plot_and_save_group_corr_heatmap(df_new, filtered_features, group_name)
    else:
        print(f"Skipping group: {group_name} — no features found in dataframe.")




In [ ]:
# Single comprehensive correlation heatmap for key features
key_features = [
    'SalePrice_capped',
    'qual_living_area_interaction',
    'Gr Liv Area_capped',
    'Garage Area',
    'total_bathrooms',
    'Total Bsmt SF',
    '1st Flr SF',
    'avg_quality',
    'Overall Cond',
    'house_age',
    'remodel_age',
    'BsmtFin SF 1',
    'Lot Area_capped'
]

# Filter features that exist in dataframe
filtered_features = [f for f in key_features if f in df_new.columns]

# Create correlation matrix
corr = df_new[filtered_features].corr()

# Create mask for upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Plot heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(
    corr,
    mask=mask,
    annot=True,
    fmt='.2f',
    cmap='RdBu_r',
    center=0,
    vmin=-1,
    vmax=1,
    square=True,
    linewidths=0.5,
    cbar_kws={'shrink': 0.8},
    annot_kws={"size": 9}
)

plt.title('Correlation Matrix: Key Predictive Features', fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()

# Save if needed
# plt.savefig('key_features_correlation_heatmap.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# As regplots for certain ordinal variables are not very informative we will replace these with boxplots 
# List of ordinal/discrete variables
boxplot_features = [
    'total_bathrooms', 'Full Bath',
    'TotRms AbvGrd', 'Bedroom AbvGr', 'Kitchen AbvGr',
    'Yr Sold', 'Fireplaces'
]


# Create output directory
output_dir = 'boxplots'
os.makedirs(output_dir, exist_ok=True)

# Loop through and create boxplots
for feature in boxplot_features:
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.boxplot(data=df_new, x=feature, y='SalePrice_capped', ax=ax, palette='pastel')
    ax.set_title(f'{feature} vs SalePrice_capped', fontsize=11)
    ax.set_xlabel(feature)
    ax.set_ylabel('SalePrice_capped')
    plt.tight_layout()

    # Save before showing
    safe_name = feature.replace('/', '_').replace('\\', '_').replace(':', '_').replace(' ', '_')
    fig.savefig(f'{output_dir}/{safe_name}_boxplot.png')
    plt.show()

#### 3.2.2 Categorical Versus Numerical

In [ ]:
# Lets look at all categorical relationships with the target
cat_cols_original = [
 'MS SubClass',
 'MS Zoning',
 'Street',
 'Lot Shape',
 'Land Contour',
 'Utilities',
 'Lot Config',
 'Land Slope',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Exter Qual',
 'Exter Cond',
 'Foundation',
 'Bsmt Qual',
 'Bsmt Cond',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'BsmtFin Type 2',
 'Heating',
 'Heating QC',
 'Central Air',
 'Electrical',
 'Kitchen Qual',
 'Functional',
 'Fireplace Qu',
 'Garage Type',
 'Garage Finish',
 'Garage Qual',
 'Garage Cond',
 'Paved Drive',
 'Sale Type',
 'Sale Condition',
 'season_sold',
 'SalePrice_bin',
 'Overall Cond'
]

# Create output directory
output_dir = 'catfeats_v_target'
os.makedirs(output_dir, exist_ok=True)

# Loop through and create boxplots
for feature in cat_cols_original:
    plt.figure(figsize=(8, 5))
    sns.boxplot(
        data=df_new,
        x='SalePrice_capped',
        y=feature,
        palette='pastel',
        orient='h'
    )
    plt.title(f'{feature} vs SalePrice_capped', fontsize=12)
    plt.xlabel('SalePrice_capped')
    plt.ylabel(feature)
    plt.tight_layout()

    # Save before showing
    safe_name = feature.replace('/', '_').replace('\\', '_').replace(':', '_').replace(' ', '_')
    plt.savefig(f'{output_dir}/{safe_name}_boxplot.png')
    plt.show()
    plt.close()  # Prevent overlap in subsequent plots

#### 3.3 Multivariate Analysis

#### 3.3.1. Correlation Matrix

In [ ]:
#Lets use our numerical features used in the written report
report_num_feats = [
    "Gr Liv Area_capped",
    "1st Flr SF",
    "Total Bsmt SF",
    "Mas Vnr Area_capped",
    "Lot Area_capped",
    "Wood Deck SF_capped",
    "total_bsmt_finished",
    "qual_living_area_interaction",
    "Full Bath",
    "total_bathrooms",
    "TotRms AbvGrd",
    "Bedroom AbvGr",
    "Kitchen AbvGr",
    "avg_quality",
    "house_age",
    "remodel_age",
    "Yr Sold",
    "Garage Area",
    "Open Porch SF_capped",
    "total_porch_area",
    "MS SubClass",
    "Fireplaces"
]

features = report_num_feats + ["SalePrice_capped"]

corr_matrix = df_new[features].corr()

# Create mask for upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(20, 10))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    vmin=-1,
    vmax=1,
    mask=mask,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.7},
    annot_kws={"size": 8}  # Smaller font size for annotation numbers
)
plt.title('Correlation Matrix of Selected Features with SalePrice_capped')
plt.tight_layout()
plt.savefig('multivariate_analysis/selected_feature_correlation_matrix_masked.png')
plt.show()

In [ ]:
#list correlations greater than .78
# Get the absolute values of correlations
abs_corr = corr_matrix.abs()

# We'll create an empty list to store tuples of (feature1, feature2, correlation)
corr_pairs = []

# Iterate over the upper triangle (to avoid duplicates and self-correlation)
for i in range(len(abs_corr.columns)):
    for j in range(i+1, len(abs_corr.columns)):
        corr_value = abs_corr.iloc[i, j]
        if corr_value >= 0.78:
            feature1 = abs_corr.columns[i]
            feature2 = abs_corr.columns[j]
            # Use the original correlation value with sign
            original_corr = corr_matrix.loc[feature1, feature2]
            corr_pairs.append((feature1, feature2, original_corr))

# Sort by absolute correlation descending
corr_pairs_sorted = sorted(corr_pairs, key=lambda x: abs(x[2]), reverse=True)

# Print nicely
for f1, f2, val in corr_pairs_sorted:
    print(f"{f1} and {f2}: {val:.3f}")

# If you want the list for further use:
# corr_pairs_sorted


In [ ]:
#list correlations between .6 and .77
# Get the absolute values of correlations
abs_corr = corr_matrix.abs()

# We'll create an empty list to store tuples of (feature1, feature2, correlation)
corr_pairs = []

# Iterate over the upper triangle (to avoid duplicates and self-correlation)
for i in range(len(abs_corr.columns)):
    for j in range(i+1, len(abs_corr.columns)):
        corr_value = abs_corr.iloc[i, j]
        if 0.6 <= corr_value < 0.78:
            feature1 = abs_corr.columns[i]
            feature2 = abs_corr.columns[j]
            # Use the original correlation value with sign
            original_corr = corr_matrix.loc[feature1, feature2]
            corr_pairs.append((feature1, feature2, original_corr))

# Sort by absolute correlation descending
corr_pairs_sorted = sorted(corr_pairs, key=lambda x: abs(x[2]), reverse=True)

# Print nicely
for f1, f2, val in corr_pairs_sorted:
    print(f"{f1} and {f2}: {val:.3f}")

# If you want the list for further use:
# corr_pairs_sorted


In [ ]:
# Clustermap
# Clean original correlation matrix by dropping all-NaN rows and columns
corr_matrix_clean = corr_matrix.dropna(axis=0, how='all').dropna(axis=1, how='all')

# Plot clustered heatmap of the cleaned top 15 correlation matrix
sns.clustermap(corr_matrix_clean, annot=True, fmt=".2f", cmap='coolwarm', figsize=(12, 12), annot_kws={"size":8})
plt.title('Clustered Correlation Matrix for Selected Features and SalePrice_capped')
plt.show()


#### 3.3.2 Grouped Boxplots (categorical vs numerical)

In [ ]:
# Categorical and numerical features
cat_feats = ['Neighborhood', 'Overall Qual', 'Garage Type', 'House Style', 'MS Zoning']
num_feats = [
    'qual_living_area_interaction', 
    'Garage Area', 
    'Total Bsmt SF',
    '1st Flr SF',
    'total_bathrooms'
]

# Folder to save plots
save_dir = 'multivariate/grouped_boxplots_sorted'
os.makedirs(save_dir, exist_ok=True)

# Loop through all combinations
for cat in cat_feats:
    for num in num_feats:
        plt.figure(figsize=(14, 6))

        # Sort categories by median of numerical feature
        order = df_new.groupby(cat)[num].median().sort_values().index

        # Plot
        sns.boxplot(data=df_new, x=cat, y=num, order=order, palette='Blues')

        plt.title(f'{num} by {cat} (sorted by median {num})')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # Show inline
       

        # Save figure
        filename = f"{save_dir}/{num}_by_{cat}_sorted.png"
        plt.savefig(filename)
        plt.show()


In [ ]:
df_new.groupby("Garage Type")["total_bathrooms"].median()

In [ ]:
df_new.groupby("House Style")["total_bathrooms"].median()

In [ ]:
df_new.groupby("MS Zoning")["total_bathrooms"].median()

## 4. Hypothesis Testing

Group qual_living_area_interaction into categories and compare sale prices.  


Null hypothesis (H0): The mean sale price is the same across all groups (Low, Medium, High).
  
Alternative hypothesis (HA): At least one group has a different mean sale price.

In [ ]:
# Step 1: Create groups
df_new['qual_group'] = pd.qcut(df_new['qual_living_area_interaction'], q=3, labels=['Low', 'Medium', 'High'])

# Step 2: Visualize
sns.boxplot(x='qual_group', y='SalePrice_capped', data=df_new)
plt.title('SalePrice_capped distribution by qual_living_area_interaction groups')


# Step 3: Extract sale prices per group
low_prices = df_new.loc[df_new['qual_group'] == 'Low', 'SalePrice_capped']
med_prices = df_new.loc[df_new['qual_group'] == 'Medium', 'SalePrice_capped']
high_prices = df_new.loc[df_new['qual_group'] == 'High', 'SalePrice_capped']

# Step 4: ANOVA test
f_stat, p_val = stats.f_oneway(low_prices, med_prices, high_prices)
print(f"ANOVA F-statistic: {f_stat}, p-value: {p_val}")

alpha = 0.05
if p_val < alpha:
    print("Reject the null hypothesis: There is a statistically significant difference in mean sale price between at least two groups of qual_living_area_interaction.")
else:
    print("Fail to reject the null hypothesis: No significant difference in sale prices across groups.")

plt.show()


In [ ]:
# Add a Kruskal–Wallis test to the hypothesis due to right-skewed distributions
kw_stat, kw_pval = kruskal(low_prices, med_prices, high_prices)
print(f"Kruskal-Wallis statistic: {kw_stat}, p-value: {kw_pval}")

### Regression Modelling Analysis

#### Model 1: Simple Linear Regression on core features


In [ ]:
df_new.columns

In [ ]:
# Your feature list and target
features = [
    'qual_living_area_interaction',
    'Gr Liv Area_capped',
    'avg_quality',
    'Garage Area',
    'total_bathrooms'
]

target = 'SalePrice_capped'

# DataFrame to store evaluation metrics
results = pd.DataFrame(columns=['Feature', 'R2', 'MSE', 'RMSE', 'MAE'])

# Set plotting style
sns.set(style="whitegrid")

# Loop through features
for i, feature in enumerate(features):
    X = df_new[[feature]]
    y = df_new[target]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    # Fit model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)

    # Metrics
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)

    # Store results
    results.loc[i] = [feature, r2, mse, rmse, mae]

    # Residuals
    residuals = y_test - y_pred

    # Plot residuals
    plt.figure(figsize=(8, 5))
    sns.scatterplot(x=y_pred, y=residuals)
    plt.axhline(0, linestyle='--', color='red')
    plt.title(f'Residuals vs Predicted Values: {feature}')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')

    # Save plot to file
    filename = f'residuals_vs_predicted_{feature.replace(" ", "_")}.png'
    plt.savefig(filename)
    plt.show()

# Show the results DataFrame
print(results.round(3))


##  5. Preprocessing for Modelling ##

In [ ]:
df_new.info()

#### 5.1 Encode ordinal categorical variables as integers ##

In [ ]:
# Separate numeric and categorical first
numeric_cols = df_new.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df_new.select_dtypes(include=['object', 'category']).columns.tolist()

print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)

# Show unique counts for each categorical col to help spot ordinal vs nominal
for col in categorical_cols:
    print(f"{col}: {df_new[col].nunique()} unique values")
    print(df_new[col].unique()[:20], "\n")  # show first 20 categories



In [ ]:
# Now manually create the ordinal list based oabove and domain knowledge
ordinal_cols = [
    'Overall Qual',
    'Overall Cond',
    'Exter Qual',
    'Exter Cond',
    'Bsmt Qual',
    'Bsmt Cond',
    'Bsmt Exposure',
    'BsmtFin Type 1',
    'BsmtFin Type 2',
    'Heating QC',
    'Kitchen Qual',
    'Functional',
    'Fireplace Qu',
    'Garage Qual',
    'Garage Cond',
    'Paved Drive',
    'qual_group'
]# e.g. quality ratings
nominal_cols = [col for col in categorical_cols if col not in ordinal_cols]

print("Ordinal columns:", ordinal_cols)
print("Nominal columns:", nominal_cols)

In [ ]:
# Encode ordinal features as integers
# Define mappings for ordinal features
ordinal_mappings = {
    'Exter Qual': {'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4},
    'Exter Cond': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'Bsmt Qual': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'Bsmt Cond': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'Bsmt Exposure': {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4},
    'BsmtFin Type 1': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
    'BsmtFin Type 2': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
    'Heating QC': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'Kitchen Qual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'Functional': {'Sal': 1, 'Sev': 2, 'Maj2': 3, 'Maj1': 4, 'Min2': 5, 'Min1': 6, 'Mod': 7, 'Typ': 8},
    'Fireplace Qu': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'Garage Qual': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'Garage Cond': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'Paved Drive': {'N': 0, 'P': 1, 'Y': 2},
    'qual_group': {'Low': 1, 'Medium': 2, 'High': 3}
}

# Apply mappings to the dataframe
for feature, mapping in ordinal_mappings.items():
    df_new[feature] = df_new[feature].map(mapping)

# Also keep Overall Qual and Overall Cond as integers (already numeric)


In [ ]:
# List of all ordinal columns
ordinal_cols = list(ordinal_mappings.keys())

# Check unique values in each ordinal column
for col in ordinal_cols:
    print(f"{col}: {df_new[col].unique()}")


#### 5.2 One-hot encode nominal features

In [ ]:
# List of nominal columns (exclude ordinal ones)
nominal_cols = [col for col in df_new.select_dtypes(include='object').columns 
                if col not in ordinal_cols]

# One-hot encode nominal columns
df_new = pd.get_dummies(df_new, columns=nominal_cols, drop_first=True)

# Quick check
print(df_new.head())


#### 5.3 Create Polynomial Features

In [ ]:
# Determine which numeric features would benefit from polynomial transformation
numeric_features = df_new.select_dtypes(include=['int64', 'float64']).columns.tolist()


# Make sure X contains only numeric features
X = df_new[numeric_features].drop(columns=['SalePrice_capped'])
y = df_new['SalePrice_capped']


results = []

for feature in X.columns:
    original_corr = np.corrcoef(X[feature], y)[0, 1]
    squared_corr = np.corrcoef(X[feature]**2, y)[0, 1]
    cubic_corr = np.corrcoef(X[feature]**3, y)[0, 1]
    
    results.append({
        "Feature": feature,
        "Original_corr": original_corr,
        "Squared_corr": squared_corr,
        "Cubic_corr": cubic_corr,
        "Squared_better": abs(squared_corr) > abs(original_corr),
        "Cubic_better": abs(cubic_corr) > abs(original_corr)
    })

df_poly_check = pd.DataFrame(results)

# Features that may benefit from polynomial transform
poly_candidates = df_poly_check[(df_poly_check['Squared_better']) | (df_poly_check['Cubic_better'])]
with pd.option_context('display.max_columns', None, 'display.width', 120):
    print(poly_candidates[['Feature', 'Original_corr', 'Squared_corr', 'Cubic_corr', 'Squared_better', 'Cubic_better']])



In [ ]:
ordinal_poly = [
    'Overall Qual', 'Overall Cond', 'Exter Qual', 'Bsmt Qual', 
    'Kitchen Qual', 'Heating QC', 'Fireplace Qu', 'Garage Qual', 
    'Paved Drive', 'BsmtFin Type 1', 'BsmtFin Type 2'
]
continuous_poly = [
    'Order', 'Garage Cars', 'Garage Yr Blt', 'Mo Sold', 'Yr Sold', 
    '2nd Flr SF', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Screen Porch_capped', 
    'living_area_ratio', 'avg_quality'
]


In [ ]:
# 1. Continuous features
poly_cont = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
cont_transformed = poly_cont.fit_transform(df_new[continuous_poly])
cont_feature_names = poly_cont.get_feature_names_out(continuous_poly)
df_cont_poly = pd.DataFrame(cont_transformed, columns=cont_feature_names, index=df_new.index)

# 2. Ordinal features
poly_ord = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
ord_transformed = poly_ord.fit_transform(df_new[ordinal_poly])
ord_feature_names = poly_ord.get_feature_names_out(ordinal_poly)
df_ord_poly = pd.DataFrame(ord_transformed, columns=ord_feature_names, index=df_new.index)

# Combine with the original dataset
df_new = pd.concat([df_new, df_cont_poly, df_ord_poly], axis=1)

# Quick checks
print(df_new.shape)
print(df_cont_poly.head())
print(df_ord_poly.head())


#### 5.4 Feature Scaling

In [ ]:
# Check columns before scaling

print(df_new.shape)
cols_to_scale = df_new.select_dtypes(include=['int64', 'float64']).columns
print(df_new[cols_to_scale].shape)
non_numeric_cols = df_new.columns.difference(cols_to_scale)
print(df_new[non_numeric_cols].shape)
print(df_new.dtypes.value_counts())

In [ ]:
# There is a disparity in the column sums to be investigated

cols_to_scale = list(df_new.select_dtypes(include=['int64', 'float64']).columns)

extra_cols = []
for col in cols_to_scale:
    series = df_new[col]  # This will be a Series if col is a scalar name
    if hasattr(series, "dtype"):  # Check that it's a Series
        if str(series.dtype) not in ['int64', 'float64']:
            extra_cols.append(col)
    else:
        # If we ever hit a multi-column name (weird case), flag it
        extra_cols.append(col)

print(extra_cols)
print(df_new[extra_cols].dtypes)


In [ ]:
# Find duplicate column names (including all occurrences, not just the 2nd+)
dup_cols = df_new.columns[df_new.columns.duplicated()].unique()

for col in dup_cols:
    # Get all columns with this name (could be more than 2)
    same_name_cols = [c for c in df_new.columns if c == col]
    
    print(f"\n--- Column name: '{col}' ---")
    display(df_new[same_name_cols].head())  # show first few rows
    # Also check if they're all identical
    identical = df_new[same_name_cols].nunique(axis=1).eq(1).all()
    print(f"Identical across duplicates? {identical}")


In [ ]:
# Remove duplicates
df_new = df_new.loc[:, ~df_new.columns.duplicated()]


In [ ]:
df_new.shape

In [ ]:
# Check columns again before scaling

print(df_new.shape)
cols_to_scale = df_new.select_dtypes(include=['int64', 'float64']).columns
print(df_new[cols_to_scale].shape)
non_numeric_cols = df_new.columns.difference(cols_to_scale)
print(df_new[non_numeric_cols].shape)
print(df_new.dtypes.value_counts())

In [ ]:
# Now I can scale at last!
# Identify numeric and non-numeric columns
numeric_cols = df_new.select_dtypes(include=['int64', 'float64']).columns
non_numeric_cols = df_new.columns.difference(numeric_cols)

# Scale numeric columns
scaler = RobustScaler()
scaled_numeric = pd.DataFrame(
    scaler.fit_transform(df_new[numeric_cols]),
    columns=numeric_cols,
    index=df_new.index
)

# Combine scaled numeric columns with non-numeric columns
df_scaled = pd.concat([scaled_numeric, df_new[non_numeric_cols]], axis=1)

# Quick check
print(df_scaled.shape)
print(df_scaled.head())


In [ ]:
# Check feature names and duplicates before proceeding to feature assessment
# List all column names
print(df_scaled.columns.tolist())

# Check for duplicates
duplicates = df_scaled.columns[df_scaled.columns.duplicated()].unique()
print("Duplicate column names:", duplicates)

# Quick preview of new polynomial feature names
poly_features = [col for col in df_scaled.columns if '^' in col or ':' in col]
print("Sample polynomial features:", poly_features[:10])


In [ ]:
# Skim features
df_scaled.columns.to_list()

#### 5.5 Feature Assessment (Post-Preprocessing)

In [ ]:
df_scaled.shape

In [ ]:
# Drop any column that starts with "Order" or contains "Order" in its name
df_scaled = df_scaled.drop(
    columns=[c for c in df_scaled.columns if "Order" in c]
)

print("Remaining columns:", len(df_scaled.columns))


In [ ]:
# Define original, created interaction terms and original capped features
original_columns = ['PID', 'Lot Frontage', 'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add', 'Exter Qual', 'Exter Cond', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Heating QC', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu', 'Garage Yr Blt', 'Garage Area', 'Garage Qual', 'Garage Cond', 'Paved Drive', 'Mo Sold', 'Yr Sold', 'Bsmt Full Bath','Bsmt Half Bath','Garage Cars','SalePrice_bin', 'living_area_inconsistency','qual_group']
interaction_columns = ['house_age', 'remodel_age', 'years_to_remodel', 'total_bathrooms', 'total_porch_area', 'total_bsmt_finished', 'living_area_ratio','avg_quality', 'qual_living_area_interaction']
capped_features = ['Lot Area_capped','Mas Vnr Area_capped','Wood Deck SF_capped','Open Porch SF_capped','Enclosed Porch_capped', '3Ssn Porch_capped','Screen Porch_capped','Pool Area_capped','Misc Val_capped','Gr Liv Area_capped','SalePrice_capped']

In [ ]:
# Define one hot encoded features
# OHE features = processed columns that weren't in the original raw dataset
exclude_cols = original_columns + interaction_columns + capped_features


ohe_features = [c for c in df_scaled.columns if df_scaled[c].nunique() == 2 and c not in exclude_cols]

poly_features = [c for c in df_scaled.columns if c not in exclude_cols + ohe_features]

print("Number of one-hot encoded features:", len(ohe_features))
print("Number of polynomial features:", len(poly_features))
print(ohe_features)
print(poly_features)
      


In [ ]:
print("Number of original features:", len(original_columns))
print("Number of interaction items:", len(interaction_columns))
print("Number of capped features:", len(capped_features))
print("Number of one-hot encoded features:", len(ohe_features))
print("Number of polynomial features:", len(poly_features))

print(df_scaled.shape)
print(len(original_columns) + len(interaction_columns) + len(capped_features) + len(ohe_features) + len(poly_features))

In [ ]:
all_features = original_columns + interaction_columns + capped_features + ohe_features + poly_features

# Find which column(s) are missing or duplicated
missing_in_df = [c for c in all_features if c not in df_scaled.columns]
extra_in_df = [c for c in df_scaled.columns if c not in all_features]

print("Columns counted but not in df_scaled:", missing_in_df)
print("Columns in df_scaled but not counted:", extra_in_df)


In [ ]:
#Check correlation with SalePrice_capped
numeric_df = df_scaled.select_dtypes(include=['number'])
corr_matrix = numeric_df.corr()
saleprice_corr = corr_matrix['SalePrice_capped']

# Filter correlations above threshold
threshold = 0.5
strong_corr = saleprice_corr[abs(saleprice_corr) > threshold].sort_values(key=abs, ascending=False)


print(strong_corr)


In [ ]:
df_scaled.dtypes.value_counts()

In [ ]:
# Check for multicolinearity

In [ ]:
# Subset to features (exclude target)
predictors = df_scaled[strong_corr.index.drop('SalePrice_capped')]

# Compute correlation matrix among predictors
pred_corr = predictors.corr().abs()

# Find pairs with correlation > 0.8
high_corr_pairs = (
    pred_corr.where(np.triu(np.ones(pred_corr.shape), k=1).astype(bool))
    .stack()
    .reset_index()
    .rename(columns={'level_0': 'Feature1', 'level_1': 'Feature2', 0: 'Correlation'})
)

high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'] > 0.8]

high_corr_pairs.sort_values('Correlation', ascending=False)


In [ ]:
# high_corr_pairs: DataFrame with columns Feature1, Feature2, Correlation
# saleprice_corr: Series of correlations of all features with SalePrice_capped

# 1. Determine which feature to keep in each highly collinear pair
high_corr_pairs['Keep_feature'] = high_corr_pairs.apply(
    lambda row: row['Feature1'] 
    if abs(saleprice_corr[row['Feature1']]) >= abs(saleprice_corr[row['Feature2']])
    else row['Feature2'],
    axis=1
)

# 2. Add correlation of the kept feature with SalePrice_capped
high_corr_pairs['Keep_feature_corr'] = high_corr_pairs['Keep_feature'].apply(lambda f: saleprice_corr[f])

# 3. Optionally sort by collinearity correlation for reference
high_corr_pairs = high_corr_pairs.sort_values('Correlation', ascending=False).reset_index(drop=True)

# 4. Create a list of features to drop (the one NOT kept in each pair)
high_corr_pairs['Drop_feature'] = high_corr_pairs.apply(
    lambda row: row['Feature2'] if row['Keep_feature'] == row['Feature1'] else row['Feature1'], axis=1
)

# 5. Full list of features to drop (no duplicates)
features_to_drop = high_corr_pairs['Drop_feature'].unique().tolist()

# 6. Optional: inspect top rows
high_corr_pairs.head()


In [ ]:
# 1. Select only numeric columns for VIF calculation
numeric_features = df_scaled.select_dtypes(include=['number']).columns.tolist()

# Optionally, remove the target column
numeric_features = [f for f in numeric_features if f != 'SalePrice_capped']

# 2. Prepare the DataFrame for VIF calculation
X_vif = df_scaled[numeric_features]

# 3. Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X_vif.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# 4. Sort by VIF descending to see the most collinear features first
vif_data = vif_data.sort_values('VIF', ascending=False).reset_index(drop=True)

# 5. Flag features with high VIF (e.g., >5)
vif_data['High_VIF'] = vif_data['VIF'] > 5

vif_data.head(20)  # Show top 20 features with highest VIF


In [ ]:
# ---------------------------
# 1. Get the features to keep
# ---------------------------

# Keep all unique features selected in 'Keep_feature'
features_to_keep = high_corr_pairs['Keep_feature'].unique().tolist()

# Also include all features that weren't part of any high-correlation pair
all_features = df_scaled.columns.tolist()
features_in_pairs = high_corr_pairs[['Feature1','Feature2']].values.flatten().tolist()
features_not_in_pairs = [f for f in all_features if f not in features_in_pairs]

# Combine
final_features = list(features_to_keep) + features_not_in_pairs

# ---------------------------
# 2. Create the trimmed DataFrame
# ---------------------------
df_trimmed = df_scaled[final_features].copy()

# Optional: check shape
print("Original df shape:", df_scaled.shape)
print("Trimmed df shape:", df_trimmed.shape)


In [ ]:
# Re-check multicollinearity (VIF + Pearson)

# --- VIF ---
# Keep only numeric columns (in case some object slipped in)
X_vif = df_trimmed.select_dtypes(include=[np.number]).copy()

# Drop the target if still present
if "SalePrice_capped" in X_vif.columns:
    X_vif = X_vif.drop(columns=["SalePrice_capped"])

# Ensure no NaNs or inf
X_vif = X_vif.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i)
                   for i in range(X_vif.shape[1])]

print("\nTop 20 features by VIF (trimmed dataset):")
print(vif_data.sort_values(by="VIF", ascending=False).head(20))

# --- Pearson Correlation ---
corr_matrix_trimmed = X_vif.corr().abs()

threshold = 0.80
high_corr_pairs_trimmed = (
    corr_matrix_trimmed.where(np.triu(np.ones(corr_matrix_trimmed.shape), k=1).astype(bool))
    .stack()
    .reset_index()
    .rename(columns={0: "Correlation", "level_0": "Feature1", "level_1": "Feature2"})
)

high_corr_pairs_trimmed = high_corr_pairs_trimmed[
    high_corr_pairs_trimmed["Correlation"] > threshold
].sort_values(by="Correlation", ascending=False)

print(f"\nNumber of highly correlated pairs > {threshold} in trimmed dataset:", 
      len(high_corr_pairs_trimmed))
print(high_corr_pairs_trimmed.head(10))



In [ ]:
# -----------------------------
# Summary of multicollinearity
# -----------------------------

# 1️⃣ High VIF features
vif_threshold = 10.0  # adjust if you like
high_vif_features = vif_data[vif_data['VIF'] > vif_threshold].copy()

print(f"Number of features with VIF > {vif_threshold}: {len(high_vif_features)}")
print(high_vif_features[['Feature','VIF']].head(20))

# 2️⃣ Highly correlated pairs (Pearson)
corr_threshold = 0.8
high_corr_pairs_summary = high_corr_pairs[high_corr_pairs['Correlation'] > corr_threshold].copy()

print(f"\nNumber of highly correlated pairs with correlation > {corr_threshold}: {len(high_corr_pairs_summary)}")
print(high_corr_pairs_summary[['Feature1','Feature2','Correlation','Keep_feature']].head(20))

# 3️⃣ Optional: combine info into one DataFrame for reference
high_corr_vif_summary = high_corr_pairs_summary.copy()
high_corr_vif_summary['Keep_feature_VIF'] = high_corr_vif_summary['Keep_feature'].map(
    dict(zip(vif_data['Feature'], vif_data['VIF']))
)
high_corr_vif_summary['Drop_feature_VIF'] = high_corr_vif_summary['Drop_feature'].map(
    dict(zip(vif_data['Feature'], vif_data['VIF']))
)

high_corr_vif_summary.head(20)


In [ ]:
# ----------------------------
# PARAMETERS
# ----------------------------
vif_threshold = 10.0
target_col = "SalePrice_capped"

# ----------------------------
# INITIAL SETUP
# ----------------------------
df_pruned = df_trimmed.copy()  # Work on trimmed dataset
numeric_features = df_pruned.select_dtypes(include=[np.number]).columns.tolist()
numeric_features = [f for f in numeric_features if f != target_col]

# Compute VIFs once
X_vif = df_pruned[numeric_features].replace([np.inf, -np.inf], np.nan).dropna(axis=1)
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# Identify features exceeding VIF threshold
high_vif_features = vif_data[vif_data["VIF"] > vif_threshold]
print(f"Number of features with VIF > {vif_threshold}: {len(high_vif_features)}")

# ----------------------------
# DECIDE WHICH FEATURES TO DROP
# ----------------------------
features_to_drop = []

if 'saleprice_corr' in globals():
    # Keep the feature most correlated with the target in each highly collinear pair
    for feature in high_vif_features["Feature"]:
        if feature not in features_to_drop:
            # Identify all features highly correlated with this one (optional: threshold 0.8)
            corr_with_others = df_pruned[numeric_features].corr()[feature].abs()
            correlated_features = corr_with_others[corr_with_others > 0.8].index.tolist()
            
            # Among these, keep the one most correlated with target
            corr_target = {f: abs(saleprice_corr.get(f, 0)) for f in correlated_features}
            feature_to_keep = max(corr_target, key=corr_target.get)
            
            # Drop the rest
            for f in correlated_features:
                if f != feature_to_keep and f not in features_to_drop:
                    features_to_drop.append(f)
else:
    # Simple approach: drop all high-VIF features
    features_to_drop = high_vif_features["Feature"].tolist()

# ----------------------------
# CREATE PRUNED DATAFRAME
# ----------------------------
df_pruned = df_pruned.drop(columns=features_to_drop)
print(f"Features removed: {len(features_to_drop)}")
print("Top 20 removed features:", features_to_drop[:20])
print("Final dataframe shape:", df_pruned.shape)



In [ ]:
df_pruned.columns.tolist()

In [ ]:
# re-Check correlation with SalePrice_capped
# Convert bools to integers for correlation
df_corr = df_pruned.copy()
bool_cols = df_corr.select_dtypes(include='bool').columns
df_corr[bool_cols] = df_corr[bool_cols].astype(int)

# Now select all numeric features including converted bools
numeric_df = df_corr.select_dtypes(include=['number'])

# Compute correlation with target
saleprice_corr = numeric_df.corr()['SalePrice_capped'].drop('SalePrice_capped').sort_values(key=abs, ascending=False)
print(saleprice_corr)
print(len(saleprice_corr))


In [ ]:
# Set correlation threshold
threshold = 0.1  # adjust as needed (e.g., 0.1, 0.3, 0.5)

# Filter features whose absolute correlation with target is above threshold
strong_corr = saleprice_corr[abs(saleprice_corr) > threshold]

# Optional: convert to DataFrame for easier inspection
strong_corr_df = pd.DataFrame({
    'Feature': strong_corr.index,
    'Correlation': strong_corr.values
}).sort_values(by='Correlation', key=abs, ascending=False)

print(strong_corr_df)
print(f"Number of features above threshold {threshold}: {len(strong_corr_df)}")


In [ ]:
# Update final df to have the top 100 features

# Get list of selected features
selected_features = strong_corr_df['Feature'].tolist()

# Ensure target is in the final dataset
if 'SalePrice_capped' not in selected_features:
    selected_features.append('SalePrice_capped')

# Create final dataset with target included
df_final = df_pruned[selected_features]

# Separate features and target for modeling
X_final = df_final.drop('SalePrice_capped', axis=1)
y_final = df_final['SalePrice_capped']


# Create final dataset
df_final = df_pruned[selected_features]



## 6. Model 1

In [ ]:
top_features = [
    'qual_living_area_interaction',
    'Overall Qual Exter Qual Fireplace Qu',
    'Overall Qual^2 BsmtFin Type 1',
    'total_bathrooms',
    'Total Bsmt SF'
]

# Split dataset once
X_all = df_final[top_features]
y_all = df_final['SalePrice_capped']

X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.2, random_state=42
)

results = []

for feature in top_features:
    # Prepare training and test sets for this feature
    X_train_feat = sm.add_constant(X_train[[feature]])
    X_test_feat = sm.add_constant(X_test[[feature]])
    
    # Fit SLR
    model = sm.OLS(y_train, X_train_feat).fit()
    
    # Predict
    y_train_pred = model.predict(X_train_feat)
    y_test_pred = model.predict(X_test_feat)
    
    # Compute metrics
    results.append({
        'Feature': feature,
        'Train R2': r2_score(y_train, y_train_pred),
        'Test R2': r2_score(y_test, y_test_pred),
        'Train RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'Test RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred)),
        'Train MAE': np.mean(np.abs(y_train - y_train_pred)),
        'Test MAE': np.mean(np.abs(y_test - y_test_pred))
    })

# Convert to DataFrame for easy viewing
results_df = pd.DataFrame(results)
results_df


In [ ]:
# Ensure directory exists to save plots
output_dir = "model1_diagnostics"
os.makedirs(output_dir, exist_ok=True)

for feature in top_features:
    # Prepare training data
    X_train_feat = sm.add_constant(X_train[[feature]])
    model = sm.OLS(y_train, X_train_feat).fit()
    
    # Residuals and fitted values
    residuals = model.resid
    fitted = model.fittedvalues
    
    # -------------------------
    # 1. Residuals vs Fitted
    # -------------------------
    plt.figure(figsize=(6,4))
    plt.scatter(fitted, residuals, alpha=0.5)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel("Fitted values")
    plt.ylabel("Residuals")
    plt.title(f"{feature}: Residuals vs Fitted")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{feature}_residuals_vs_fitted.png")  # Save first
    plt.show()
    plt.close()
    
    # -------------------------
    # 2. QQ plot
    # -------------------------
    fig = sm.qqplot(residuals, line='45', fit=True)
    plt.title(f"{feature}: QQ Plot")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{feature}_qqplot.png")
    plt.show()
    plt.close()
    
    # -------------------------
    # 3. Cook's distance
    # -------------------------
    influence = model.get_influence()
    cooks_d, _ = influence.cooks_distance
    
    plt.figure(figsize=(6,4))
    plt.stem(range(len(cooks_d)), cooks_d, markerfmt=",")
    plt.xlabel("Observation index")
    plt.ylabel("Cook's distance")
    plt.title(f"{feature}: Cook's Distance")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{feature}_cooks_distance.png")
    plt.show()
    plt.close()
    
    # -------------------------
    # 4. Influence plot
    # -------------------------
    fig, ax = plt.subplots(figsize=(8,6))
    sm.graphics.influence_plot(model, ax=ax, criterion="cooks")
    plt.title(f"{feature}: Influence Plot")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{feature}_influence_plot.png")
    plt.show()
    plt.close()
    
    # -------------------------
    # 5. Normality tests
    # -------------------------
    shapiro_test = stats.shapiro(residuals)
    jb_test = sm.stats.stattools.jarque_bera(residuals)
    
    print(f"{feature} Diagnostics:")
    print(f"  Shapiro-Wilk test: W={shapiro_test[0]:.4f}, p={shapiro_test[1]:.4f}")
    print(f"  Jarque-Bera test: JB={jb_test[0]:.4f}, p={jb_test[1]:.4f}")
    print("-"*50)


## 7. Model 2

In [ ]:
# --------------------------
# Define scorers for CV
# --------------------------
scoring = {
    'r2': make_scorer(r2_score),
    'rmse': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))),
    'mae': make_scorer(mean_absolute_error)
}

# --------------------------
# Features & target
# --------------------------
# Drop problematic / leakage columns
X = df_final.drop('SalePrice_capped', axis=1)

# Target
y = df_final['SalePrice_capped']


# --------------------------
# 5-fold CV with Multiple Linear Regression
# --------------------------
linreg = LinearRegression()

kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_results = cross_validate(
    linreg, X, y,
    cv=kf,
    scoring=scoring,
    return_train_score=True
)

# --------------------------
# Summarise CV results
# --------------------------
results_df = pd.DataFrame({
    'Train R2': cv_results['train_r2'],
    'Test R2': cv_results['test_r2'],
    'Train RMSE': cv_results['train_rmse'],
    'Test RMSE': cv_results['test_rmse'],
    'Train MAE': cv_results['train_mae'],
    'Test MAE': cv_results['test_mae']
})

print("Cross-validation results (per fold):")
print(results_df)

print("\nAverage performance across folds:")
print(results_df.mean())


In [ ]:
# Visualise CV results
# Extract per-fold values
folds = np.arange(1, 6)
train_r2 = results_df['Train R2']
test_r2 = results_df['Test R2']
train_rmse = results_df['Train RMSE']
test_rmse = results_df['Test RMSE']
train_mae = results_df['Train MAE']
test_mae = results_df['Test MAE']

output_dir = "model2"
os.makedirs(output_dir, exist_ok=True)

# ---------- R² ----------
plt.figure(figsize=(8,4))
plt.bar(folds - 0.15, train_r2, width=0.3, label='Train R²')
plt.bar(folds + 0.15, test_r2, width=0.3, label='Test R²')
plt.xlabel("Fold")
plt.ylabel("R²")
plt.title("Cross-validation R² per Fold")
plt.xticks(folds)
plt.ylim(0,1)
plt.legend()
plt.tight_layout()
plt.savefig(f"{output_dir}/cv_r2_per_fold.png")
plt.show()
plt.close()

# ---------- RMSE ----------
plt.figure(figsize=(8,4))
plt.plot(folds, train_rmse, marker='o', label='Train RMSE')
plt.plot(folds, test_rmse, marker='o', label='Test RMSE')
plt.xlabel("Fold")
plt.ylabel("RMSE")
plt.title("Cross-validation RMSE per Fold")
plt.xticks(folds)
plt.legend()
plt.tight_layout()
plt.savefig(f"{output_dir}/cv_rmse_per_fold.png")
plt.show()
plt.close()

# ---------- MAE ----------
plt.figure(figsize=(8,4))
plt.plot(folds, train_mae, marker='o', label='Train MAE')
plt.plot(folds, test_mae, marker='o', label='Test MAE')
plt.xlabel("Fold")
plt.ylabel("MAE")
plt.title("Cross-validation MAE per Fold")
plt.xticks(folds)
plt.legend()
plt.tight_layout()
plt.savefig(f"{output_dir}/cv_mae_per_fold.png")
plt.show()
plt.close()

In [ ]:
# Lets look at the important features

# --------------------------
# Fit Linear Regression on full data
# --------------------------
linreg.fit(X, y)

# --------------------------
# Feature importance (coefficients)
# --------------------------
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': linreg.coef_
})

top_model_2_features = feature_importance_df.reindex(
    feature_importance_df['Coefficient'].abs().sort_values(ascending=False).index
).head(10)

print(top_model_2_features)


## 8. Iterative Refinement

#### 8.1 Outlier Treatment

In [ ]:
# --------------------------
# Setup output folder
# --------------------------
out_dir = "outlier_treatment_29_08_25"
os.makedirs(out_dir, exist_ok=True)

# --------------------------
# Prepare data for OLS (full-sample)
# --------------------------
X_sm = df_final.drop('SalePrice_capped', axis=1).copy()

# Convert boolean columns to int, drop near-constant ones
bool_cols = X_sm.select_dtypes(include='bool').columns
for col in bool_cols:
    freq = X_sm[col].value_counts(normalize=True).max()
    if freq > 0.95:
        X_sm.drop(columns=col, inplace=True)
    else:
        X_sm[col] = X_sm[col].astype(int)

# Ensure all numeric
X_sm = X_sm.apply(pd.to_numeric, errors='coerce')
X_sm = sm.add_constant(X_sm)

# Target
y_sm = pd.to_numeric(df_final['SalePrice_capped'], errors='coerce')

# Drop any rows with NaN
mask = X_sm.notna().all(axis=1) & y_sm.notna()
X_sm = X_sm.loc[mask]
y_sm = y_sm.loc[mask]

# --------------------------
# Fit full-sample OLS
# --------------------------
ols_cook_distance1 = sm.OLS(y_sm, X_sm).fit()

# Influence measures
influence = ols_cook_distance1.get_influence()
cooks_d, _ = influence.cooks_distance
student_resid = influence.resid_studentized_external
leverage = influence.hat_matrix_diag

# --------------------------
# Identify influential points
# --------------------------
n = X_sm.shape[0]
threshold = 4/n
influential_points = np.where(cooks_d > threshold)[0]

# Select top 10 automatically
top_10_indices = np.argsort(cooks_d)[-10:][::-1]
top_10_index_values = X_sm.index[top_10_indices]

top_10_df = pd.DataFrame({
    'index': top_10_index_values,
    "Cook's Distance": cooks_d[top_10_indices],
    'target': y_sm.loc[top_10_index_values].values
})

print(f"Threshold for Cook's distance: {threshold:.4f}")
print(f"Number of influential points: {len(influential_points)}")
print("Top 10 influential points (index, Cook's D, target):")
print(top_10_df)

# --------------------------
# Bubble plot: leverage vs studentized residuals
# --------------------------
fig, ax = plt.subplots(figsize=(8,6))
bubble_size = 500 * (cooks_d / np.nanmax(cooks_d))
ax.scatter(leverage, student_resid, s=bubble_size, alpha=0.5)

# Convert to Series for indexed selection
leverage_series = pd.Series(leverage, index=X_sm.index)
student_resid_series = pd.Series(student_resid, index=X_sm.index)

# Highlight top 10
ax.scatter(leverage_series.loc[top_10_index_values],
           student_resid_series.loc[top_10_index_values],
           color='red', s=100, label='Top influencers')

ax.axhline(y=0, color='grey', linestyle='--')
ax.set_xlabel("Leverage")
ax.set_ylabel("Studentized Residuals")
ax.set_title("Influence Plot (Bubble Size = Cook's Distance)")
ax.legend()
plt.tight_layout()
plt.savefig(f"{out_dir}/influence_plot.png")
plt.show()


In [ ]:
#This appears unstable so checking VIF
# Drop constant column if present
X_numeric = X_sm.drop(columns='const', errors='ignore')

# Only numeric predictors
X_numeric = X_numeric.select_dtypes(include=np.number)

# Compute VIFs
vif_data = pd.DataFrame()
vif_data['feature'] = X_numeric.columns
vif_data['VIF'] = [variance_inflation_factor(X_numeric.values, i)
                   for i in range(X_numeric.shape[1])]

# Sort by VIF descending
vif_data = vif_data.sort_values('VIF', ascending=False)

# Show high VIFs (commonly > 5 or > 10)
high_vif = vif_data[vif_data['VIF'] > 5]

print("Features with high multicollinearity (VIF > 5):")
print(high_vif)


In [ ]:
# Step 1: Create dummies
X_sm = pd.get_dummies(
    df_final.drop(columns='SalePrice_capped', axis=1),
    drop_first=True
)

# Step 2: Force numeric float type
X_sm = X_sm.astype(float)

# Step 3: Drop any rows with NaN or inf
X_sm = X_sm.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

# Step 4: Drop near-constant columns
near_constant_cols = [col for col in X_sm.columns 
                      if X_sm[col].value_counts(normalize=True).max() > 0.95]
X_sm.drop(columns=near_constant_cols, inplace=True)

# Step 5: Optional: remove any remaining object columns just in case
X_sm = X_sm.select_dtypes(include=[np.number])

# Step 6: Compute VIF
vif_data = pd.DataFrame()
vif_data['feature'] = X_sm.columns
vif_data['VIF'] = [variance_inflation_factor(X_sm.values, i) 
                   for i in range(X_sm.shape[1])]
vif_data = vif_data.sort_values('VIF', ascending=False)

print("Top 20 VIFs:")
print(vif_data.head(20))


In [ ]:
# Drop redundant numeric columns
numeric_drop = ['Full Bath', 'Half Bath', 'Bsmt Full Bath']
X_sm = X_sm.drop(columns=numeric_drop)

#Drop redundant dummy columns
for col in X_sm.columns:
    if X_sm[col].value_counts(normalize=True).max() > 0.95:
        X_sm.drop(columns=col, inplace=True)

# Drop one of any pair of perfectly correlated columns
corr_matrix = X_sm.corr().abs()
high_corr_pairs = [(corr_matrix.columns[i], corr_matrix.columns[j])
                   for i, j in zip(*np.where(corr_matrix>0.99)) if i<j]
for col1, col2 in high_corr_pairs:
    X_sm.drop(columns=col2, inplace=True)

# Ensure all columns are numeric and no NaNs
X_sm = X_sm.apply(pd.to_numeric, errors='coerce')
X_sm = X_sm.dropna(axis=1, how='all')
X_sm = X_sm.dropna(axis=0)

# Recompute VIFs

vif_data = pd.DataFrame()
vif_data['feature'] = X_sm.columns
vif_data['VIF'] = [variance_inflation_factor(X_sm.values, i) for i in range(X_sm.shape[1])]
vif_data = vif_data.sort_values('VIF', ascending=False)
print(vif_data.head(20))


In [ ]:
# Further VIF clearning needed

drop_features = [
    'years_to_remodel',
    'Wood Deck SF_capped', 'Open Porch SF_capped',
    'Exterior 1st_MetalSd', 'Exterior 2nd_MetalSd',
    'Garage Type_Detchd',
    'Garage Cars Mo Sold avg_quality', 'Garage Cars Bsmt Unf SF avg_quality',
    'Roof Style_Gable',
    'Sale Condition_Partial'
]

X_sm_cleaned = X_sm.drop(columns=[f for f in drop_features if f in X_sm.columns])

# Recalculate VIFs after cleaning
def calculate_vif(df):
    return pd.DataFrame({
        'feature': df.columns,
        'VIF': [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    }).sort_values('VIF', ascending=False)

vif_df_cleaned = calculate_vif(X_sm_cleaned)
print("Top VIFs after manual grouping cleanup:")
print(vif_df_cleaned.head(20))


In [ ]:
# Now let's try to run cook's Distance again

# --------------------------
# Setup output folder
# --------------------------
out_dir = "outlier_treatment_29_08_25"
os.makedirs(out_dir, exist_ok=True)

# --------------------------
# Use your cleaned X_sm and y_sm
# --------------------------
# X_sm: your VIF-cleaned dataframe (numeric, dummies, no near-constant columns)
# y_sm: df_pruned['SalePrice_capped'], numeric
X_sm = X_sm_cleaned
# Ensure numeric and add constant
X_sm = sm.add_constant(X_sm)
y_sm = pd.to_numeric(y_sm, errors='coerce')

# Drop rows with any NaN (just in case)
mask = X_sm.notna().all(axis=1) & y_sm.notna()
X_sm = X_sm.loc[mask]
y_sm = y_sm.loc[mask]

# --------------------------
# Fit OLS
# --------------------------
ols_cook_distance2 = sm.OLS(y_sm, X_sm).fit()

# --------------------------
# Influence measures
# --------------------------
influence = ols_cook_distance2.get_influence()
cooks_d, _ = influence.cooks_distance
student_resid = influence.resid_studentized_external
leverage = influence.hat_matrix_diag

# --------------------------
# Flag influential points
# --------------------------
n = X_sm.shape[0]
threshold = 4/n
influential_points = np.where(cooks_d > threshold)[0]

print(f"Threshold for Cook's distance: {threshold:.4f}")
print(f"Number of influential points: {len(influential_points)}")

# Get top 10 influential points
top_10_indices = np.argsort(cooks_d)[-10:][::-1]
top_10_df = pd.DataFrame({
    'index': X_sm.index[top_10_indices],
    "Cook's Distance": cooks_d[top_10_indices],
    'target': y_sm.iloc[top_10_indices].values
})
print("Top 10 influential points (index, Cook's D, target):")
print(top_10_df)

# --------------------------
# Bubble plot: leverage vs studentized residuals
# --------------------------
fig, ax = plt.subplots(figsize=(8,6))
bubble_size = 500 * (cooks_d / np.nanmax(cooks_d))
ax.scatter(leverage, student_resid, s=bubble_size, alpha=0.5)

# Convert arrays to Series for indexed selection
leverage_series = pd.Series(leverage, index=X_sm.index)
student_resid_series = pd.Series(student_resid, index=X_sm.index)

# Highlight top 10 influencers
ax.scatter(leverage_series.loc[top_10_df['index']],
           student_resid_series.loc[top_10_df['index']],
           color='red', s=100, label='Top influencers')

ax.axhline(y=0, color='grey', linestyle='--')
ax.set_xlabel("Leverage")
ax.set_ylabel("Studentized Residuals")
ax.set_title("Influence Plot (Bubble Size = Cook's Distance)")
ax.legend()
plt.tight_layout()
plt.savefig(f"{out_dir}/influence_plot.png")
plt.show()


In [ ]:
# Lets look at cooks distance once more alongside key diagnostics for (as yet unchanged) Model2 to provide baseline

# Output folder
out_dir = "outlier_treatment3_29_08_25"
os.makedirs(out_dir, exist_ok=True)

# Prepare data (VIF-cleaned, numeric)
X_infl = X_sm.apply(pd.to_numeric, errors='coerce')
y_infl = pd.to_numeric(y_sm, errors='coerce')

# Drop NaN / Inf rows
mask = X_infl.notna().all(axis=1) & np.isfinite(X_infl).all(axis=1) & y_infl.notna() & np.isfinite(y_infl)
X_infl = X_infl.loc[mask]
y_infl = y_infl.loc[mask]

# Add constant
X_infl = sm.add_constant(X_infl)

# Fit OLS
ols_cook_distance4 = sm.OLS(y_infl, X_infl).fit()

# Influence measures
influence = ols_cook_distance4.get_influence()
cooks_d, _ = influence.cooks_distance
student_resid = influence.resid_studentized_external
leverage = influence.hat_matrix_diag

# Identify top influencers
threshold = 4 / X_infl.shape[0]
influential_points = np.where(cooks_d > threshold)[0]

top_10_idx = np.argsort(cooks_d)[-10:][::-1]
top_10_df = pd.DataFrame({
    'index': X_infl.index[top_10_idx],
    "Cook's Distance": cooks_d[top_10_idx],
    'target': y_infl.iloc[top_10_idx].values
})

print(f"Cook's distance threshold: {threshold:.4f}")
print(f"Number of influential points: {len(influential_points)}")
print("Top 10 influential points:")
print(top_10_df)

# Bubble plot
plt.figure(figsize=(8,6))
bubble_size = 500 * (cooks_d / np.nanmax(cooks_d))
plt.scatter(leverage, student_resid, s=bubble_size, alpha=0.5)
plt.scatter(leverage[top_10_idx], student_resid[top_10_idx], color='red', s=100, label='Top influencers')
plt.axhline(0, color='grey', linestyle='--')
plt.xlabel("Leverage")
plt.ylabel("Studentized Residuals")
plt.title("Influence Plot (Cook's Distance)")
plt.legend()
plt.tight_layout()
plt.savefig(f"{out_dir}/influence_plot.png")
plt.show()

# MODEL 2 EVALUATION

# Features & target (same as model 2)
X_cv = df_final.drop('SalePrice_capped', axis=1)

# Target
y_cv = df_final['SalePrice_capped']

# Scorers for CV
scoring = {
    'r2': make_scorer(r2_score),
    'rmse': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))),
    'mae': make_scorer(mean_absolute_error)
}

linreg = LinearRegression()
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_results = cross_validate(linreg, X_cv, y_cv, cv=kf, scoring=scoring, return_train_score=True)

results_df2 = pd.DataFrame({
    'Train R2': cv_results['train_r2'],
    'Test R2': cv_results['test_r2'],
    'Train RMSE': cv_results['train_rmse'],
    'Test RMSE': cv_results['test_rmse'],
    'Train MAE': cv_results['train_mae'],
    'Test MAE': cv_results['test_mae']
})

print("Cross-validation results (per fold):")
print(results_df2)
print("\nAverage performance across folds:")
print(results_df2.mean())

# Residual Diagnostics for model 2
# Fit on full data for residual plots
linreg.fit(X_cv, y_cv)
y_pred = linreg.predict(X_cv)
residuals = y_cv - y_pred

# Residuals vs Fitted
plt.figure(figsize=(8,6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, color='grey', linestyle='--')
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (Model 2)")
plt.tight_layout()
plt.savefig(f"{out_dir}/residuals_vs_fitted_model2.png") 
plt.show()

# QQ Plot
plt.figure(figsize=(6,6))
sm.qqplot(residuals, line='45', fit=True)
plt.title("QQ Plot of Residuals (Model 2)")
plt.tight_layout()
plt.savefig("f"{out_dir}/qq_plot_model2.png")
plt.show()

# Shapiro-Wilk test
from scipy import stats
shapiro_p = stats.shapiro(residuals)[1]
print(f"Shapiro-Wilk p-value (Model 2 residuals): {shapiro_p:.4f}")


In [ ]:
# Keep only numeric columns
X_vif = X_cv.select_dtypes(include=[np.number]).copy()

vif_data = pd.DataFrame()
vif_data['feature'] = X_vif.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) 
                   for i in range(X_vif.shape[1])]

# Round and sort
vif_data['VIF'] = vif_data['VIF'].round(3)
vif_data_sorted = vif_data.sort_values('VIF', ascending=False).reset_index(drop=True)

print(vif_data_sorted.head(10))





###### Summary

1. Residuals vs Fitted
Elliptical but denser on the left → mild heteroscedasticity.
Slight spread at higher fitted values, with ~5 outliers between 1–4 (fitted) and -1.5 to -3.8 (residuals), and ~10 points between 3–4 (fitted) and -0.5 to 0.5 (residuals). Overall pattern typical for skewed real estate targets.

2. QQ Plot & Shapiro-Wilk
QQ plot: Most points near the line at upper quantiles, slight right skew. Lower quantiles more spread than before, two extreme points at (-3, -13).
Shapiro-Wilk p-value ≈ 0 → residuals not perfectly normal. Large dataset → minor deviations unlikely to harm prediction.

3. Cook’s Distance
Approximately 156 points exceed the 4/n threshold (0.0014), indicating a sizable number of potentially influential observations.

Top Cook’s D values are dominated by extreme residuals or leverage points; the largest Cook’s D is ≈ 0.887.

The most influential points (e.g., indices 1498, 2180, 2181) may disproportionately affect coefficient estimates, suggesting the need to investigate or possibly Winsorize/extensively check these rows.

Overall, while many points are flagged, the majority have moderate influence (Cook’s D < 0.05), so the model is not dominated by a few extreme cases.

4. VIFs
Several features exhibit very high multicollinearity: years_to_remodel, remodel_age, and house_age have VIF = ∞, indicating perfect or near-perfect linear dependence.

Other notable high-VIF features include total_bathrooms (50.77), total_porch_area (41.14), Wood Deck SF_capped (29.44), and Full Bath (26.13), highlighting remaining multicollinearity concerns.

Features with VIF > 10 (e.g., Bsmt Full Bath, qual_living_area_interaction, Garage Cars Mo Sold avg_quality) suggest some coefficients may be unstable or inflated.

Despite preprocessing and feature pruning, some collinearity persists; careful interpretation of coefficients is advised, and future regularization may help stabilize the model.


Implication: Many features remain highly collinear, with several showing VIFs well above 10 and some even infinite. For predictive purposes, this may be tolerable, but coefficient estimates for these variables are unstable. If interpretability becomes a priority, consider combining, dropping, or transforming highly collinear columns.


###### Interpretation

Model R² > 90% already → likely any removal of points will not improve predictive accuracy much.

Cook’s distance shows influential points, but removing hundreds can destroy model structure

Residuals indicate some skew and heteroscedasticity → could consider robust regression or transformation (e.g., log(y)) in future projects, but out of scope if sticking to vanilla linear regression.

In [ ]:
# Suppose top_cooks_indices is a list or array of indices of top Cook's distances
top_cooks_indices = [1498, 2180, 2181, 1182, 2737, 2570, 1945, 1782, 91, 2666]

# Define the columns you want to display
cols_to_show = ['Gr Liv Area_capped', 'Total Bsmt SF', 'Overall Qual', 'Overall Cond', 
                'SalePrice_capped', 'Year Built', 'Year Remod/Add', 'Lot Area_capped', 
                'BsmtFin SF 1', 'BsmtFin SF 2', 'Garage Cars', 'Garage Area']

# Select the rows and only these columns
top_cooks_original = df_new.loc[top_cooks_indices, cols_to_show]

# Display them nicely
display(
    top_cooks_original.style
    .set_table_attributes("style='display:inline'")  # inline display
    .set_table_styles([{'selector': 'th', 'props': [('font-size', '12pt')]}])  # header style
)


Let's check for anything unusual in these rows.  

1498: Huge home, top quality, but extremely low sale price → likely data error or extreme discount.

2180: Very large home, high quality, low sale price → potential outlier.

2181: Large home, high quality, low price → unusual, similar concern as 2180.

1182: Large home, high quality but poor condition → low price plausible but extreme.

2737: Large home, moderate quality but very high condition → price relatively high for quality.

2570: Large home, decent quality, low basement area → sale price slightly high for features.

1945: Large home, modest quality, minimal basement → price seems inconsistent, potential outlier.

1782: Smaller home, decent quality → low sale price plausible, less suspicious.

91: Medium-large home, good quality, basement and garage modest → sale price reasonable.

2666: Large, top-quality, very old home → sale price high but consistent with features.


In [ ]:
# Compare identified indices to their medians and IQRs

# Flagged row indices
flagged_indices = [1498, 2180, 2181, 1182, 2737, 2570, 1945, 1782, 91, 2666]

# Key columns to check
key_cols = ['Gr Liv Area_capped', 'Total Bsmt SF', 'Overall Qual', 'Overall Cond', 'SalePrice_capped',
            'Year Built', 'Year Remod/Add', 'Lot Area_capped', 'BsmtFin SF 1', 'BsmtFin SF 2', 
            'Garage Cars', 'Garage Area']

# Subset flagged rows
flagged_rows = df_new.loc[flagged_indices, key_cols]

# Compute median and IQR for the full dataset
median_vals = df_new[key_cols].median()
q1 = df_new[key_cols].quantile(0.25)
q3 = df_new[key_cols].quantile(0.75)

# Combine into a comparison DataFrame
comparison_df = flagged_rows.copy()
for col in key_cols:
    comparison_df[f'{col}_median'] = median_vals[col]
    comparison_df[f'{col}_IQR_low'] = q1[col]
    comparison_df[f'{col}_IQR_high'] = q3[col]

comparison_df


1498: Extremely large home and basement, top quality, but sale price at the lower IQR → potential data entry error.

2180: Very large home, high quality, sale price below median → suspiciously low; likely outlier.

2181: Large home, high quality, price slightly above 2180 but still below expected range → possible outlier.

1182: Large home, high quality, very small basement, price well below median → unusual, may be an outlier.

2737: Large home, moderate quality, very high condition, sale price above median → plausible, less extreme.

2570: Large home, decent quality, low basement and garage space, sale price slightly above median → mildly unusual but not extreme.

1945: Large home, low quality, minimal basement, price above median → inconsistent, potential outlier.

1782: Smaller home, decent quality, all features within median/IQR range → reasonable, likely not an outlier.

91: Medium-large home, good quality, basement and garage modest, price above median → plausible, not extreme.

2666: Large, top-quality, very old home, sale price at upper end → unusual but could be valid for historic property.

Most likely worth removing (extreme potential outliers):

1498 – huge size, top quality, extremely low sale price.

2180 – very large and high-quality, sale price unusually low.

2181 – similar to 2180; large and high-quality, sale price lower than expected.

1182 – very large, high quality, extremely small basement, very low price.

The others are generally consistent with the dataset when compared to median/IQR.

In [ ]:
# --------------------------
# Original flagged row indices (positional in df_pruned)
# --------------------------
flagged_indices = [1498, 2181, 2180, 1182]

# --------------------------
# Cross-validation setup
# --------------------------
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Scorers
scoring = {
    'r2': make_scorer(r2_score),
    'rmse': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))),
    'mae': make_scorer(mean_absolute_error)
}

# --------------------------
# Original model2 metrics
# --------------------------
# Features & target (same as model 2)
X_cv = df_final.drop('SalePrice_capped', axis=1)

# Target
y_cv = df_final['SalePrice_capped']

original_cv = cross_validate(LinearRegression(), X_cv, y_cv, cv=kf, scoring=scoring, return_train_score=True)
original_scores = {
    'train_r2': np.mean(original_cv['train_r2']),
    'test_r2': np.mean(original_cv['test_r2']),
    'train_rmse': np.mean(original_cv['train_rmse']),
    'test_rmse': np.mean(original_cv['test_rmse']),
    'train_mae': np.mean(original_cv['train_mae']),
    'test_mae': np.mean(original_cv['test_mae'])
}

print("Original model performance (average across folds):")
for k, v in original_scores.items():
    print(f"{k}: {v:.3f}")
print("\nRunning outlier sensitivity analysis...\n")

# --------------------------
# Outlier sensitivity analysis
# --------------------------
results = []

for r in range(len(flagged_indices)+1):
    for subset in combinations(flagged_indices, r):
        # Drop rows by index if any
        if subset:
            df_subset = df_final.drop(index=list(subset))
        else:
            df_subset = df_final.copy()

        X = df_subset.drop('SalePrice_capped', axis=1)
        y = df_subset['SalePrice_capped']

        cv = cross_validate(LinearRegression(), X, y, cv=kf, scoring=scoring, return_train_score=True)

        # Record results with deltas
        results.append({
            'removed_indices': subset,
            'test_r2': np.mean(cv['test_r2']),
            'delta_test_r2': np.mean(cv['test_r2']) - original_scores['test_r2'],
            'test_rmse': np.mean(cv['test_rmse']),
            'delta_test_rmse': np.mean(cv['test_rmse']) - original_scores['test_rmse'],
            'test_mae': np.mean(cv['test_mae']),
            'delta_test_mae': np.mean(cv['test_mae']) - original_scores['test_mae']
        })

# --------------------------
# Format results for readability
# --------------------------
outlier_removal_results_df = pd.DataFrame(results)

# Round numbers for clarity
for col in ['test_r2','delta_test_r2','test_rmse','delta_test_rmse','test_mae','delta_test_mae']:
    outlier_removal_results_df[col] = outlier_removal_results_df[col].round(3)

# Sort by delta_test_r2 descending (best improvements on top)
outlier_removal_results_df = outlier_removal_results_df.sort_values('delta_test_r2', ascending=False).reset_index(drop=True)

print("Outlier sensitivity analysis results (compared to original model):")
print(outlier_removal_results_df.to_string(index=False))


Verdict

These four rows are true high-leverage / influential points.

They are distorting the model, likely because their size, basement, and price are inconsistent relative to the rest of the data.

Removing all four clearly improves cross-validated performance.

✅ Recommendation: Remove these four rows before final modeling.

In [ ]:
# =============================
# 0. Create output folder
# =============================
output_folder = "Outlier treatment iteration"
os.makedirs(output_folder, exist_ok=True)

# =============================
# 1. Create a new df without the outliers
# =============================
df_outliers_removed = df_final.drop(flagged_indices, errors="ignore")

# =============================
# 2. Define predictors and response
# =============================
X = df_outliers_removed.drop('SalePrice_capped', axis=1)
# Convert categories and bools to numeric
for col in X.columns:
    if pd.api.types.is_categorical_dtype(X[col]) or pd.api.types.is_bool_dtype(X[col]):
        X[col] = X[col].astype(int)
y = df_outliers_removed["SalePrice_capped"]

# Convert to numeric and drop any remaining NaNs
X = X.apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(y, errors='coerce')
X = X.dropna()
y = y.loc[X.index]

# Add constant for statsmodels
X_sm = sm.add_constant(X)

# =============================
# 3. Fit final model
# =============================
model_outliers_removed = sm.OLS(y, X_sm).fit()

# =============================
# 4. Performance metrics
# =============================
print("Model Performance (with outliers removed):")
print(model_outliers_removed.summary())

# =============================
# 5. Residual diagnostics
# =============================
residuals = model_outliers_removed.resid
fitted = model_outliers_removed.fittedvalues

# QQ plot
qq_plot_path = os.path.join(output_folder, "QQ_Plot_Outliers_Removed.png")
sm.qqplot(residuals, line='45', fit=True)
plt.title("QQ Plot (Outliers Removed)")
plt.savefig(qq_plot_path)
plt.show()  # <-- display
plt.close()

# Residuals vs Fitted
residuals_plot_path = os.path.join(output_folder, "Residuals_vs_Fitted_Outliers_Removed.png")
plt.scatter(fitted, residuals, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (Outliers Removed)")
plt.savefig(residuals_plot_path)
plt.show()  # <-- display
plt.close()

# Cook's Distance
# Cook's distance & influence plot after outlier removal
influence = model_outliers_removed.get_influence()
cooks_d, _ = influence.cooks_distance
student_resid = influence.resid_studentized_external
leverage = influence.hat_matrix_diag

# Threshold
n, p = X_sm.shape
threshold = 4 / n

# Identify influential points
influential_points = np.where(cooks_d > threshold)[0]

# Top 10 by Cook's D
top_10_idx = np.argsort(cooks_d)[-10:][::-1]
top_10_df = pd.DataFrame({
    "index": X.index[top_10_idx],
    "Cook's Distance": cooks_d[top_10_idx],
    "target": y.iloc[top_10_idx].values
})

print(f"Cook's distance threshold: {threshold:.4f}")
print(f"Number of influential points: {len(influential_points)}")
print("Top 10 influential points:")
print(top_10_df)

# Bubble plot (scaled by Cook's D)
plt.figure(figsize=(8,6))
bubble_size = 500 * (cooks_d / np.nanmax(cooks_d))
plt.scatter(leverage, student_resid, s=bubble_size, alpha=0.5)
plt.scatter(leverage[top_10_idx], student_resid[top_10_idx],
            color="red", s=100, label="Top influencers")
plt.axhline(0, color="grey", linestyle="--")
plt.xlabel("Leverage")
plt.ylabel("Studentized Residuals")
plt.title("Influence Plot (Cook's Distance, Outliers Removed)")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "Influence_BubblePlot_Outliers_Removed.png"))
plt.show()

# Shapiro-Wilk test
shapiro_test = stats.shapiro(residuals)
print(f"Shapiro-Wilk test: Statistic={shapiro_test.statistic:.4f}, p-value={shapiro_test.pvalue:.4f}")

# =============================
# 6. VIF
# =============================
X_vif = X_sm.select_dtypes(include=[np.number]).drop(columns='const', errors='ignore')

vif_data = pd.DataFrame()
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) 
                   for i in range(X_vif.shape[1])]

print("\nVariance Inflation Factors (Outliers Removed):")
print(vif_data.sort_values(by="VIF", ascending=False).head(10))

# =============================
# 7. Cross-Validation Performance (per fold)
# =============================
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Scoring functions
scoring = {
    'r2': make_scorer(r2_score),
    'rmse': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))),
    'mae': make_scorer(mean_absolute_error)
}

# Run cross-validation
cv_results = cross_validate(LinearRegression(), X, y, cv=kf, scoring=scoring, return_train_score=True)

# Create DataFrame with per-fold results
cv_df = pd.DataFrame({
    'Train R2': cv_results['train_r2'],
    'Test R2': cv_results['test_r2'],
    'Train RMSE': cv_results['train_rmse'],
    'Test RMSE': cv_results['test_rmse'],
    'Train MAE': cv_results['train_mae'],
    'Test MAE': cv_results['test_mae']
})

print("Cross-validation results (per fold):")
print(cv_df)

# Compute average performance across folds
avg_performance = cv_df.mean().rename("Average performance across folds")
print("\nAverage performance across folds:")
print(avg_performance)

# =============================
# 8. Save cross-validation results and VIF to CSV
# =============================
cv_df.to_csv(os.path.join(output_folder, "CV_results_Outliers_Removed.csv"), index=False)
vif_data.to_csv(os.path.join(output_folder, "VIF_Outliers_Removed.csv"), index=False)


In [ ]:
# Lets look at the important features

# --------------------------
# Fit Linear Regression on full data
# --------------------------
linreg.fit(X, y)

# --------------------------
# Feature importance (coefficients)
# --------------------------
outliers_removed_feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': linreg.coef_
})

outliers_removed_model_features = outliers_removed_feature_importance_df.reindex(
    outliers_removed_feature_importance_df['Coefficient'].abs().sort_values(ascending=False).index
).head(10)

print(outliers_removed_model_features)


#### 8.2 Feature Refinement and Response Transformation

In [ ]:

# =============================
# 0. Create output folder
# =============================
output_folder_yeo = "Outlier treatment iteration Yeo"
os.makedirs(output_folder_yeo, exist_ok=True)

# =============================
# 1. df without outliers (reuse existing df)
# =============================
df_outliers_removed = df_final.drop(flagged_indices, errors="ignore")

# =============================
# 2. Define predictors and response
# =============================
X = df_outliers_removed.drop('SalePrice_capped', axis=1)
for col in X.columns:
    if pd.api.types.is_categorical_dtype(X[col]) or pd.api.types.is_bool_dtype(X[col]):
        X[col] = X[col].astype(int)
X = X.apply(pd.to_numeric, errors='coerce')
X = X.dropna()

y = df_outliers_removed.loc[X.index, "SalePrice_capped"]

# =============================
# 2b. Apply Yeo-Johnson transform to response
# =============================
pt = PowerTransformer(method='yeo-johnson', standardize=False)
y_yeo = pt.fit_transform(y.values.reshape(-1, 1)).flatten()

# =============================
# 3. Add constant for statsmodels
# =============================
X_sm = sm.add_constant(X)

# =============================
# 4. Fit model with transformed response
# =============================
model_yeo = sm.OLS(y_yeo, X_sm).fit()
print("OLS Summary (Yeo-Johnson transformed response):")
print(model_yeo.summary())

# =============================
# 5. Residual diagnostics
# =============================
residuals_yeo = model_yeo.resid
fitted_yeo = model_yeo.fittedvalues

# QQ plot
qq_plot_path = os.path.join(output_folder_yeo, "QQ_Plot_Yeo.png")
sm.qqplot(residuals_yeo, line='45', fit=True)
plt.title("QQ Plot (Yeo-Johnson)")
plt.savefig(qq_plot_path)
plt.show()
plt.close()

# Residuals vs Fitted
residuals_plot_path = os.path.join(output_folder_yeo, "Residuals_vs_Fitted_Yeo.png")
plt.scatter(fitted_yeo, residuals_yeo, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (Yeo-Johnson)")
plt.savefig(residuals_plot_path)
plt.show()
plt.close()

# Cook's distance & influence
influence = model_yeo.get_influence()
cooks_d, _ = influence.cooks_distance
student_resid = influence.resid_studentized_external
leverage = influence.hat_matrix_diag
n, p = X_sm.shape
threshold = 4 / n
influential_points = np.where(cooks_d > threshold)[0]

# Top 10 influencers
top_10_idx = np.argsort(cooks_d)[-10:][::-1]
top_10_df = pd.DataFrame({
    "index": X.index[top_10_idx],
    "Cook's Distance": cooks_d[top_10_idx],
    "target": y.iloc[top_10_idx].values
})
print(f"Cook's distance threshold: {threshold:.4f}")
print(f"Number of influential points: {len(influential_points)}")
print("Top 10 influential points:")
print(top_10_df)

# Influence bubble plot
plt.figure(figsize=(8,6))
bubble_size = 500 * (cooks_d / np.nanmax(cooks_d))
plt.scatter(leverage, student_resid, s=bubble_size, alpha=0.5)
plt.scatter(leverage[top_10_idx], student_resid[top_10_idx],
            color="red", s=100, label="Top influencers")
plt.axhline(0, color="grey", linestyle="--")
plt.xlabel("Leverage")
plt.ylabel("Studentized Residuals")
plt.title("Influence Plot (Yeo-Johnson)")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_folder_yeo, "Influence_BubblePlot_Yeo.png"))
plt.show()

# Shapiro-Wilk test
shapiro_test = stats.shapiro(residuals_yeo)
print(f"Shapiro-Wilk test: Statistic={shapiro_test.statistic:.4f}, p-value={shapiro_test.pvalue:.4f}")

# =============================
# 6. VIF
# =============================
X_vif = X_sm.select_dtypes(include=[np.number]).drop(columns='const', errors='ignore')
vif_data = pd.DataFrame({
    "feature": X_vif.columns,
    "VIF": [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
})
print("\nVariance Inflation Factors (Yeo-Johnson):")
print(vif_data.sort_values(by="VIF", ascending=False).head(10))

# =============================
# 7. Cross-validation performance
# =============================
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    'r2': make_scorer(r2_score),
    'rmse': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))),
    'mae': make_scorer(mean_absolute_error)
}

cv_results_yeo = cross_validate(LinearRegression(), X, y_yeo, cv=kf, scoring=scoring, return_train_score=True)
cv_df_yeo = pd.DataFrame({
    'Train R2': cv_results_yeo['train_r2'],
    'Test R2': cv_results_yeo['test_r2'],
    'Train RMSE': cv_results_yeo['train_rmse'],
    'Test RMSE': cv_results_yeo['test_rmse'],
    'Train MAE': cv_results_yeo['train_mae'],
    'Test MAE': cv_results_yeo['test_mae']
})
print("Cross-validation results (Yeo-Johnson, per fold):")
print(cv_df_yeo)

avg_performance_yeo = cv_df_yeo.mean().rename("Average performance across folds")
print("\nAverage performance across folds (Yeo-Johnson):")
print(avg_performance_yeo)

# Save CV and VIF
cv_df_yeo.to_csv(os.path.join(output_folder_yeo, "CV_results_Yeo.csv"), index=False)
vif_data.to_csv(os.path.join(output_folder_yeo, "VIF_Yeo.csv"), index=False)


In [ ]:
# Make a copy of the outliers-removed df
df_yeo = df_outliers_removed.copy()
df_yeo['SalePrice_capped_YJ'] = y_yeo

# Check
df_yeo[['SalePrice_capped', 'SalePrice_capped_YJ']].head()



In [ ]:
# Lets look at the important features

# --------------------------
# Fit Linear Regression on full data
# --------------------------
linreg.fit(X, y_yeo)

# --------------------------
# Feature importance (coefficients)
# --------------------------
yeo_feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': linreg.coef_
})

yeo_model_features = yeo_feature_importance_df.reindex(
    yeo_feature_importance_df['Coefficient'].abs().sort_values(ascending=False).index
).head(10)

print(yeo_model_features)


#### 8.3 Stepwise Feature Selection

In [ ]:
# =============================
# 0. Output folder
# =============================
output_folder = "Stepwise_Iteration"
os.makedirs(output_folder, exist_ok=True)

# =============================
# 1. Prepare data
# =============================
X = df_yeo.drop(['SalePrice_capped_YJ','SalePrice_capped'], axis=1).copy()
y = df_yeo['SalePrice_capped_YJ'].copy()

# Encode categorical/bool features as int
for col in X.columns:
    if pd.api.types.is_categorical_dtype(X[col]) or pd.api.types.is_bool_dtype(X[col]):
        X[col] = X[col].astype(int)

X = X.apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(y, errors='coerce')
X = X.dropna()
y = y.loc[X.index]

# Add constant for statsmodels
X_sm = sm.add_constant(X)

# =============================
# 2. Stepwise Feature Selection
# =============================
def stepwise_selection(X, y, criterion='aic', verbose=True):
    """Perform forward-backward stepwise selection using AIC or BIC."""
    initial_features = []
    best_features = initial_features.copy()
    remaining_features = list(X.columns)
    current_score, best_new_score = np.inf, np.inf

    while remaining_features:
        scores_with_candidates = []
        for candidate in remaining_features:
            features_to_test = best_features + [candidate]
            model = sm.OLS(y, sm.add_constant(X[features_to_test])).fit()
            score = model.aic if criterion.lower() == 'aic' else model.bic
            scores_with_candidates.append((score, candidate))

        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates[0]

        if current_score > best_new_score:
            remaining_features.remove(best_candidate)
            best_features.append(best_candidate)
            current_score = best_new_score
            if verbose:
                print(f"Add {best_candidate}: {criterion.upper()}={best_new_score:.4f}")
        else:
            break

    return best_features

# Run stepwise for AIC and BIC
selected_aic = stepwise_selection(X, y, criterion='aic')
selected_bic = stepwise_selection(X, y, criterion='bic')

# =============================
# 3. Fit final models on full dataset (for diagnostics)
# =============================
model_aic = sm.OLS(y, sm.add_constant(X[selected_aic])).fit()
model_bic = sm.OLS(y, sm.add_constant(X[selected_bic])).fit()

# =============================
# 4. Residual Diagnostics & Plots
# =============================
def plot_diagnostics(model, name):
    residuals = model.resid
    fitted = model.fittedvalues

    # Residuals vs Fitted
    plt.figure(figsize=(8,6))
    plt.scatter(fitted, residuals, alpha=0.5)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel("Fitted values")
    plt.ylabel("Residuals")
    plt.title(f"Residuals vs Fitted ({name})")
    plt.savefig(os.path.join(output_folder, f"Residuals_vs_Fitted_{name}.png"))
    plt.show()
    plt.close()

    # QQ plot
    sm.qqplot(residuals, line='45', fit=True)
    plt.title(f"QQ Plot ({name})")
    plt.savefig(os.path.join(output_folder, f"QQ_Plot_{name}.png"))
    plt.show()
    plt.close()

    # Shapiro-Wilk
    shapiro_test = stats.shapiro(residuals)
    print(f"{name} - Shapiro-Wilk: Stat={shapiro_test.statistic:.4f}, p={shapiro_test.pvalue:.4e}")

    # Cook's Distance & Influence
    influence = model.get_influence()
    cooks_d, _ = influence.cooks_distance
    leverage = influence.hat_matrix_diag
    student_resid = influence.resid_studentized_external
    threshold = 4 / n
    influential_points = np.where(cooks_d > threshold)[0]


    # Top 10 influential points
    top_10_idx = np.argsort(cooks_d)[-10:][::-1]
    top_10_df = pd.DataFrame({
        "index": X.index[top_10_idx],
        "Cook's Distance": cooks_d[top_10_idx],
        "target": y.iloc[top_10_idx].values
    })
    print(f"Cook's distance threshold: {threshold:.4f}")
    print(f"Number of influential points: {len(influential_points)}")
    print(f"Top 10 influential points ({name}):\n", top_10_df)
    
    plt.figure(figsize=(8,6))
    bubble_size = 500 * (cooks_d / np.nanmax(cooks_d))
    plt.scatter(leverage, student_resid, s=bubble_size, alpha=0.5)
    plt.scatter(leverage[top_10_idx], student_resid[top_10_idx],
            color="red", s=100, label="Top influencers")
    plt.axhline(0, color='grey', linestyle='--')
    plt.xlabel("Leverage")
    plt.ylabel("Studentized Residuals")
    plt.title(f"Influence Bubble Plot ({name})")
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f"Influence_BubblePlot_{name}.png"))
    plt.show()
    plt.close()

   

    # VIF
    X_vif = sm.add_constant(X[selected_aic]).select_dtypes(include=[np.number]).drop(columns='const', errors='ignore')
    vif_data = pd.DataFrame({
        "feature": X_vif.columns,
        "VIF": [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
    })
    print(f"Variance Inflation Factors ({name}):\n", vif_data.sort_values(by="VIF", ascending=False).head(10))
    vif_data.to_csv(os.path.join(output_folder, f"VIF_{name}.csv"), index=False)

plot_diagnostics(model_aic, "Stepwise_AIC")
plot_diagnostics(model_bic, "Stepwise_BIC")

# =============================
# 5. Cross-Validation Performance (use previous folds!)
# =============================
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Use your existing CV splits if stored

scoring = {
    'r2': make_scorer(r2_score),
    'rmse': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))),
    'mae': make_scorer(mean_absolute_error)
}

cv_results_aic = cross_validate(LinearRegression(), X[selected_aic], y, cv=kf, scoring=scoring, return_train_score=True)
cv_results_bic = cross_validate(LinearRegression(), X[selected_bic], y, cv=kf, scoring=scoring, return_train_score=True)

def summarize_cv(cv_results, name):
    cv_df = pd.DataFrame({
        'Train R2': cv_results['train_r2'],
        'Test R2': cv_results['test_r2'],
        'Train RMSE': cv_results['train_rmse'],
        'Test RMSE': cv_results['test_rmse'],
        'Train MAE': cv_results['train_mae'],
        'Test MAE': cv_results['test_mae']
    })
    print(f"Cross-validation results ({name}):\n", cv_df)
    avg_performance = cv_df.mean().rename("Average performance across folds")
    print(f"\nAverage performance ({name}):\n", avg_performance)
    cv_df.to_csv(os.path.join(output_folder, f"CV_results_{name}.csv"), index=False)

summarize_cv(cv_results_aic, "Stepwise_AIC")
summarize_cv(cv_results_bic, "Stepwise_BIC")


In [ ]:
# =============================
# New dataframe for BIC model
# =============================

# Features and target restricted to BIC
X_bic = X[selected_bic].copy()
y_bic = y.copy()

# Combine into a single dataframe
BIC_df = X_bic.copy()
BIC_df['SalePrice_capped_YJ'] = y_bic

print(f"BIC_df shape: {BIC_df.shape}")
BIC_df.head()


In [ ]:
# Lets look at the important features

# --------------------------
# Fit Linear Regression on full data
# --------------------------
linreg.fit(X_bic, y_bic)

# --------------------------
# Feature importance (coefficients)
# --------------------------
BIC_importance_df = pd.DataFrame({
    'Feature': X_bic.columns,
    'Coefficient': linreg.coef_
})

bic_model_features = BIC_importance_df.reindex(
    BIC_importance_df['Coefficient'].abs().sort_values(ascending=False).index
).head(10)

print(bic_model_features)


#### 8.4 Regularization (Ridge / Lasso / Elastic Net)

In [ ]:
# =============================
# 0. Output folder
# =============================
output_folder = "Regularization_Models"
os.makedirs(output_folder, exist_ok=True)

# =============================
# 1. Prepare data
# =============================
X = BIC_df.drop('SalePrice_capped_YJ', axis=1).copy()
y = BIC_df['SalePrice_capped_YJ'].copy()

# Standardize numeric features for regularization
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

# =============================
# 2. Cross-validation setup
# =============================
kf = KFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    'r2': make_scorer(r2_score),
    'rmse': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))),
    'mae': make_scorer(mean_absolute_error)
}

# =============================
# 3. Hyperparameter grids
# =============================
ridge_params = {'alpha': np.logspace(-3, 3, 10)}
lasso_params = {'alpha': np.logspace(-3, 1, 10)}
elastic_params = {'alpha': np.logspace(-3, 1, 10), 'l1_ratio': [0.2, 0.5, 0.8]}

# =============================
# 4. Fit models with GridSearchCV
# =============================
def fit_model(model, params, X, y, cv):
    gs = GridSearchCV(model, params, cv=cv, scoring='r2', n_jobs=-1)
    gs.fit(X, y)
    best_model = gs.best_estimator_
    print(f"{model.__class__.__name__} Best Params: {gs.best_params_}")
    return best_model

ridge_best = fit_model(Ridge(), ridge_params, X_scaled, y, kf)
lasso_best = fit_model(Lasso(max_iter=10000), lasso_params, X_scaled, y, kf)
elastic_best = fit_model(ElasticNet(max_iter=10000), elastic_params, X_scaled, y, kf)

# =============================
# 5. Residual Diagnostics
# =============================
def plot_residuals(model, X, y, name):
    y_pred = model.predict(X)
    residuals = y - y_pred
    
    # Residuals vs Fitted
    plt.figure(figsize=(8,6))
    plt.scatter(y_pred, residuals, alpha=0.5)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel("Fitted values")
    plt.ylabel("Residuals")
    plt.title(f"Residuals vs Fitted ({name})")
    plt.savefig(os.path.join(output_folder, f"Residuals_vs_Fitted_{name}.png"))
    plt.show()
    plt.close()
    
    # QQ Plot
    sm.qqplot(residuals, line='45', fit=True)
    plt.title(f"QQ Plot ({name})")
    plt.savefig(os.path.join(output_folder, f"QQ_{name}.png"))
    plt.show()
    plt.close()
    
    # Shapiro-Wilk
    shapiro_test = stats.shapiro(residuals)
    print(f"{name} - Shapiro-Wilk: Stat={shapiro_test.statistic:.4f}, p={shapiro_test.pvalue:.4e}")
    
    # Cook's distance using OLS on residuals
    X_sm = sm.add_constant(X)
    ols_model = sm.OLS(y, X_sm).fit()
    influence = ols_model.get_influence()
    cooks_d, _ = influence.cooks_distance
    leverage = influence.hat_matrix_diag
    student_resid = influence.resid_studentized_external
    threshold = 4 / n
    influential_points = np.where(cooks_d > threshold)[0]


    # Top 10 influential points
    top_10_idx = np.argsort(cooks_d)[-10:][::-1]
    top_10_df = pd.DataFrame({
        "index": X.index[top_10_idx],
        "Cook's Distance": cooks_d[top_10_idx],
        "target": y.iloc[top_10_idx].values
    })
    print(f"Cook's distance threshold: {threshold:.4f}")
    print(f"Number of influential points: {len(influential_points)}")
    print(f"Top 10 influential points ({name}):\n", top_10_df)
    
    # Influence plot (bubble)
    plt.figure(figsize=(8,6))
    bubble_size = 500 * (cooks_d / np.nanmax(cooks_d))
    plt.scatter(leverage, student_resid, s=bubble_size, alpha=0.5)
    plt.scatter(leverage[top_10_idx], student_resid[top_10_idx], color="red", s=100, label="Top influencers")
    plt.axhline(0, color='grey', linestyle='--')
    plt.xlabel("Leverage")
    plt.ylabel("Studentized Residuals")
    plt.title(f"Influence Bubble Plot ({name})")
    plt.tight_layout()
    plt.legend()
    plt.savefig(os.path.join(output_folder, f"Influence_Bubble_{name}.png"))
    plt.show()
    plt.close()
    
    # VIF
    X_vif = sm.add_constant(X).select_dtypes(include=[np.number]).drop(columns='const', errors='ignore')
    vif_data = pd.DataFrame({
        "feature": X_vif.columns,
        "VIF": [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
    })
    print(f"Variance Inflation Factors ({name}):\n", vif_data.sort_values(by="VIF", ascending=False).head(10))
    vif_data.to_csv(os.path.join(output_folder, f"VIF_{name}.csv"), index=False)

# Apply to all models
plot_residuals(ridge_best, X_scaled, y, "Ridge")
plot_residuals(lasso_best, X_scaled, y, "Lasso")
plot_residuals(elastic_best, X_scaled, y, "ElasticNet")

# =============================
# 6. Cross-Validation Performance
# =============================
def cross_val_metrics(model, X, y, cv, name):
    cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=True)
    cv_df = pd.DataFrame({
        'Train R2': cv_results['train_r2'],
        'Test R2': cv_results['test_r2'],
        'Train RMSE': cv_results['train_rmse'],
        'Test RMSE': cv_results['test_rmse'],
        'Train MAE': cv_results['train_mae'],
        'Test MAE': cv_results['test_mae']
    })
    print(f"Cross-validation results ({name}):\n", cv_df)
    avg_perf = cv_df.mean().rename("Average performance across folds")
    print(f"\nAverage performance ({name}):\n", avg_perf)
    cv_df.to_csv(os.path.join(output_folder, f"CV_results_{name}.csv"), index=False)

cross_val_metrics(ridge_best, X_scaled, y, kf, "Ridge")
cross_val_metrics(lasso_best, X_scaled, y, kf, "Lasso")
cross_val_metrics(elastic_best, X_scaled, y, kf, "ElasticNet")


In [ ]:
# Lets look at the important features
ridge_importance_df = pd.DataFrame({
    'Feature': X_scaled.columns,
    'Coefficient': ridge_best.coef_
})

ridge_model_features = ridge_importance_df.reindex(
    ridge_importance_df['Coefficient'].abs().sort_values(ascending=False).index
).head(10)

print(ridge_model_features)
